In [ ]:
# agg
from pathlib import Path
import json

def aggregate_code_cells(input_ipynb: str, output_ipynb: str) -> Path:
    """
    Aggregate all code cells from a Jupyter notebook into a new notebook
    with a single combined code cell.

    Parameters
    ----------
    input_ipynb : str
        Path to the source .ipynb file.
    output_ipynb : str
        Path to the new .ipynb file to create.

    Returns
    -------
    Path
        The path to the written aggregated notebook.
    """
    input_path = Path(input_ipynb).expanduser().resolve()
    output_path = Path(output_ipynb).expanduser()

    if not input_path.is_file():
        raise FileNotFoundError(f"Input notebook not found: {input_path}")

    # --- 1) Load the source notebook ----------------------------------------
    with input_path.open("r", encoding="utf-8") as f:
        nb = json.load(f)

    cells = nb.get("cells", [])
    nbformat = nb.get("nbformat", 4)
    nbformat_minor = nb.get("nbformat_minor", 5)

    # --- 2) Collect all code from code cells --------------------------------
    aggregated_source_lines = []

    for idx, cell in enumerate(cells):
        if cell.get("cell_type") != "code":
            continue

        src = cell.get("source", [])
        if isinstance(src, str):
            src_lines = [src]
        else:
            src_lines = src

        # Mark original cell boundaries (optional but helpful)
        aggregated_source_lines.append(f"# ===== Cell {idx} =====\n")
        aggregated_source_lines.extend(src_lines)

        # Ensure a blank line between chunks
        if not aggregated_source_lines[-1].endswith("\n"):
            aggregated_source_lines.append("\n")
        aggregated_source_lines.append("\n")

    if not aggregated_source_lines:
        aggregated_source_lines = ["# No code cells found in source notebook.\n"]

    # --- 3) Build the new notebook structure --------------------------------
    aggregated_cell = {
        "cell_type": "code",
        "metadata": {},
        "source": aggregated_source_lines,
        "outputs": [],
        "execution_count": None,
    }

    new_nb = {
        "cells": [aggregated_cell],
        "metadata": {
            "aggregated_from": str(input_path),
        },
        "nbformat": nbformat,
        "nbformat_minor": nbformat_minor,
    }

    # --- 4) Ensure parent dir exists & write the new notebook ---------------
    output_path.parent.mkdir(parents=True, exist_ok=True)
    with output_path.open("w", encoding="utf-8") as f:
        json.dump(new_nb, f, ensure_ascii=False, indent=1)

    print(f"✅ Aggregated notebook written to: {output_path.resolve()}")
    return output_path.resolve()


In [ ]:
# AGGREGATE_PYTHON_CELLS_SCRIPT
# Report Aggregator
#!/usr/bin/env python3

"""
Aggregate all Python code cells from a source notebook into a new notebook
stored one level up in a 'reports' folder, with:
- section-based grouping (1.1, 1.2, etc.)
- exactly ONE aggregated code cell per section group
- rich metadata + TODO summary
- integrity checks to ensure no cells are lost or duplicated
"""

import json
import sys
import platform
import re
from pathlib import Path
from datetime import datetime

import nbformat
from nbformat.v4 import new_notebook, new_markdown_cell

# CONFIG

# Absolute path to the source notebook
NOTEBOOK_PATH = Path("/Users/b/DATA/PROJECTS/Telco/Level_3/notebooks/01_EDA.ipynb")

# Timestamp for filenames and metadata
START_TIME_UTC = datetime.utcnow()
TS_STR = START_TIME_UTC.strftime("%Y%m%d_%H%M%S")

# Output directory: one level up from "notebooks", into "reports"
OUTPUT_DIR = NOTEBOOK_PATH.parent.parent.parent / "reports"
OUTPUT_NOTEBOOK_NAME = f"AGG_ALL_{TS_STR}.ipynb"
OUTPUT_NOTEBOOK_PATH = OUTPUT_DIR / OUTPUT_NOTEBOOK_NAME
OUTPUT_METADATA_JSON = OUTPUT_DIR / f"AGG_ALL_{TS_STR}_metadata.json"

# HELPERS

def load_notebook(path: Path):
    if not path.exists():
        raise FileNotFoundError(f"Source notebook not found: {path}")
    return nbformat.read(path, as_version=4)

def ensure_output_dir(path: Path):
    path.mkdir(parents=True, exist_ok=True)


def parse_section_header(md_source: str):
    """
    Try to extract a numbered section tag from a markdown cell.

    Looks for patterns like:
      '# 1.1 Title', '## 2.3.4 Something', '1.2 Another title', etc.

    Returns dict:
        {
          "group_key": "1.1",      # top-level or first two components
          "full_tag": "1.1.1",     # full numeric string
          "title": "My Section",   # title text (if present)
        }
    or None if no numbered header is found.
    """
    for line in md_source.splitlines():
        stripped = line.strip()
        if not stripped:
            continue

        # Remove leading '#' heading markers if present
        text = re.sub(r'^#+\s*', '', stripped)

        # Match numeric tag at the start: 1.1, 2.3.4, etc.
        m = re.match(r'(?P<tag>\d+(?:\.\d+)+)\s*(?P<title>.*)', text)
        if not m:
            continue

        full_tag = m.group("tag")
        title = (m.group("title") or "").strip() or f"Section {full_tag}"

        parts = full_tag.split(".")
        if len(parts) >= 2:
            group_key = ".".join(parts[:2])  # group by 1.1, 1.2, etc.
        else:
            group_key = full_tag

        return {
            "group_key": group_key,
            "full_tag": full_tag,
            "title": title,
        }

    return None


def extract_code_cells_and_groups(nb):
    """
    Extract all non-empty code cells, tracking:
      - section grouping (based on nearest numbered markdown header)
      - TODO/FIXME comments
      - integrity stats

    Returns:
        section_groups (dict): mapping group_key -> section info:
            {
                group_key: {
                    "title": str,
                    "cells": [
                        {
                            "source": str,
                            "source_index": int,
                            "todo_lines": [str],
                            "char_count": int,
                        },
                        ...
                    ],
                    "first_source_index": int,
                    "full_tags": set([...]),
                },
                ...
            }
        stats (dict): global stats for code extraction
        todo_items (list[str]): list of TODO/FIXME lines (with cell indices)
    """
    section_groups = {}

    total_code = 0
    non_empty_code = 0
    empty_code = 0
    non_empty_char_count = 0
    todo_items = []

    current_group_key = "UNSECTIONED"
    current_section_title = "Unsectioned / No numbered header yet"
    current_full_tag = None

    for idx, cell in enumerate(nb.cells):
        cell_type = cell.get("cell_type")

        # Update section context when we hit numbered markdown headers
        if cell_type == "markdown":
            info = parse_section_header(cell.get("source", "") or "")
            if info:
                current_group_key = info["group_key"]
                current_section_title = info["title"]
                current_full_tag = info["full_tag"]
            continue

        if cell_type != "code":
            continue

        total_code += 1
        source = cell.get("source", "") or ""
        if not source.strip():
            empty_code += 1
            continue

        non_empty_code += 1
        non_empty_char_count += len(source)

        # Collect TODO/FIXME lines
        cell_todos = []
        for line in source.splitlines():
            stripped = line.strip()
            if "TODO" in stripped or "FIXME" in stripped:
                cell_todos.append(stripped)

        if cell_todos:
            todo_items.extend([f"Cell {idx}: {line}" for line in cell_todos])

        # Ensure a group exists for the current section
        if current_group_key not in section_groups:
            section_groups[current_group_key] = {
                "title": current_section_title,
                "cells": [],
                "first_source_index": idx,
                "full_tags": set(),
            }

        grp = section_groups[current_group_key]
        # Update first_source_index if this is earlier
        if grp["first_source_index"] is None or idx < grp["first_source_index"]:
            grp["first_source_index"] = idx
        # Track full tags
        if current_full_tag:
            grp["full_tags"].add(current_full_tag)

        # Append cell info
        grp["cells"].append(
            {
                "source": source,
                "source_index": idx,
                "todo_lines": cell_todos,
                "char_count": len(source),
            }
        )

    # Sanity check: total code vs (empty + non-empty)
    assert total_code == empty_code + non_empty_code, (
        f"Code cell accounting mismatch: total={total_code}, "
        f"empty={empty_code}, non_empty={non_empty_code}"
    )

    stats = {
        "total_code_cells_found": total_code,
        "non_empty_code_cells_extracted": non_empty_code,
        "empty_code_cells_skipped": empty_code,
        "non_empty_code_char_count_source": non_empty_char_count,
        "section_group_count": len(section_groups),
    }
    return section_groups, stats, todo_items


def build_metadata(nb, code_stats, todo_items, section_groups):
    """
    Build a detailed metadata dict about the aggregation run.
    (Also computes destination cell counts and integrity checks.)
    """
    total_cells = len(nb.cells)
    markdown_cells = sum(1 for c in nb.cells if c.get("cell_type") == "markdown")
    raw_cells = sum(1 for c in nb.cells if c.get("cell_type") == "raw")
    other_cells = total_cells - markdown_cells - raw_cells - code_stats["total_code_cells_found"]

    # Section breakdown list
    section_breakdown = []
    for group_key, grp in sorted(
        section_groups.items(),
        key=lambda kv: kv[1]["first_source_index"],
    ):
        section_breakdown.append(
            {
                "section_group": group_key,
                "title": grp["title"],
                "code_cells_in_group": len(grp["cells"]),
                "first_source_index": grp["first_source_index"],
                "full_tags": sorted(t for t in grp["full_tags"] if t),
            }
        )

    # Destination stats: 2 metadata cells + (header + code) per section
    n_groups = len(section_groups)
    dest_stats = {
        "total_cells_in_destination": 2 + 2 * n_groups,
        "code_cells_in_destination": n_groups,          # 1 per section
        "metadata_cells_in_destination": 2,             # metadata + TODO
        "section_header_cells_in_destination": n_groups,
    }

    # Integrity checks: ensure every non-empty code cell appears exactly once in a group
    all_indices = []
    for grp in section_groups.values():
        for cell_info in grp["cells"]:
            all_indices.append(cell_info["source_index"])
    unique_indices = set(all_indices)

    integrity_checks = {
        "source_non_empty_code_cells": code_stats["non_empty_code_cells_extracted"],
        "assigned_to_groups_total": len(all_indices),
        "unique_source_indices_in_groups": len(unique_indices),
        "all_source_code_cells_assigned_exactly_once": (
            len(all_indices) == len(unique_indices) == code_stats["non_empty_code_cells_extracted"]
        ),
        "non_empty_code_char_count_source": code_stats["non_empty_code_char_count_source"],
    }

    meta = {
        "run_timestamp_utc": START_TIME_UTC.isoformat() + "Z",
        "run_timestamp_local": datetime.now().isoformat(),
        "source_notebook": str(NOTEBOOK_PATH),
        "output_notebook": str(OUTPUT_NOTEBOOK_PATH),
        "output_metadata_json": str(OUTPUT_METADATA_JSON),
        "environment": {
            "python_version": sys.version,
            "platform": platform.platform(),
            "executable": sys.executable,
        },
        "notebook_structure": {
            "total_cells_in_source": total_cells,
            "code_cells_in_source": code_stats["total_code_cells_found"],
            "markdown_cells_in_source": markdown_cells,
            "raw_cells_in_source": raw_cells,
            "other_cells_in_source": other_cells,
        },
        "aggregation_stats": {
            # Scrape counts
            "total_code_cells_in_source": code_stats["total_code_cells_found"],
            "non_empty_code_cells_scraped_from_source": code_stats["non_empty_code_cells_extracted"],
            "empty_code_cells_skipped": code_stats["empty_code_cells_skipped"],
            # Destination counts
            "total_cells_created_in_destination": dest_stats["total_cells_in_destination"],
            "code_cells_in_destination": dest_stats["code_cells_in_destination"],
            "metadata_cells_in_destination": dest_stats["metadata_cells_in_destination"],
            "section_header_cells_in_destination": dest_stats["section_header_cells_in_destination"],
            "section_group_count": code_stats["section_group_count"],
        },
        "todo_summary": {
            "todo_items_count": len(todo_items),
            "todo_items_preview": todo_items[:20],
        },
        "section_breakdown": section_breakdown,
        "integrity_checks": integrity_checks,
        "tooling": {
            "agg_script_name": "aggregate_python_cells.py",
            "version": "3.0.0",
            "description": (
                "Aggregates Python/code cells from source notebook into a clean "
                "notebook with exactly one Python cell per numbered section."
            ),
        },
    }
    return meta


def build_metadata_markdown(metadata: dict) -> str:
    """
    Build a human-readable markdown summary of the metadata.
    """
    nb_struct = metadata["notebook_structure"]
    agg_stats = metadata["aggregation_stats"]
    env = metadata["environment"]
    integrity = metadata["integrity_checks"]
    sections = metadata["section_breakdown"]

    md = []
    md.append("# 📦 Aggregated Python Cells Notebook")
    md.append("")
    md.append("This notebook was auto-generated by a Python aggregation script.")
    md.append("")
    md.append("## 🕒 Run Info")
    md.append(f"- **Run UTC:** `{metadata['run_timestamp_utc']}`")
    md.append(f"- **Run Local:** `{metadata['run_timestamp_local']}`")
    md.append(f"- **Source Notebook:** `{metadata['source_notebook']}`")
    md.append(f"- **Output Notebook:** `{metadata['output_notebook']}`")
    md.append("")
    md.append("## 🧱 Source Notebook Structure")
    md.append(f"- Total cells: **{nb_struct['total_cells_in_source']}**")
    md.append(f"- Code cells: **{nb_struct['code_cells_in_source']}**")
    md.append(f"- Markdown cells: **{nb_struct['markdown_cells_in_source']}**")
    md.append(f"- Raw cells: **{nb_struct['raw_cells_in_source']}**")
    md.append(f"- Other cells: **{nb_struct['other_cells_in_source']}**")
    md.append("")
    md.append("## 🧮 Aggregation Stats")
    md.append(f"- Code cells scraped from source (non-empty): "
              f"**{agg_stats['non_empty_code_cells_scraped_from_source']}**")
    md.append(f"- Empty code cells skipped: **{agg_stats['empty_code_cells_skipped']}**")
    md.append("")
    md.append(f"- Cells created in destination notebook: "
              f"**{agg_stats['total_cells_created_in_destination']}**")
    md.append(f"  - Code cells (one per section): **{agg_stats['code_cells_in_destination']}**")
    md.append(f"  - Metadata cells: **{agg_stats['metadata_cells_in_destination']}**")
    md.append(f"  - Section header cells: **{agg_stats['section_header_cells_in_destination']}**")
    md.append("")
    md.append("## 📚 Section Grouping Overview")
    md.append(f"- Number of section groups: **{agg_stats['section_group_count']}**")
    md.append("")
    if sections:
        for sec in sections:
            sec_title = sec["title"]
            sec_key = sec["section_group"]
            n_cells = sec["code_cells_in_group"]
            tags = sec["full_tags"]
            if tags:
                tag_str = ", ".join(tags)
                md.append(f"- **{sec_key}** — {sec_title} "
                          f"(original code cells merged: **{n_cells}**, tags: `{tag_str}`)")
            else:
                md.append(f"- **{sec_key}** — {sec_title} "
                          f"(original code cells merged: **{n_cells}**)")
    else:
        md.append("_No numbered sections detected; all cells are grouped under 'UNSECTIONED'._")
    md.append("")
    md.append("## 🧪 Integrity Checks")
    md.append(
        f"- Non-empty code cells in source: "
        f"**{integrity['source_non_empty_code_cells']}**"
    )
    md.append(
        f"- Total assignments into section groups: "
        f"**{integrity['assigned_to_groups_total']}**"
    )
    md.append(
        f"- Unique source code cell indices in groups: "
        f"**{integrity['unique_source_indices_in_groups']}**"
    )
    md.append(
        f"- All non-empty code cells assigned exactly once: "
        f"**{'✅ Yes' if integrity['all_source_code_cells_assigned_exactly_once'] else '❌ No (investigate)'}**"
    )
    md.append(
        f"- Total characters across non-empty code cells (source): "
        f"**{integrity['non_empty_code_char_count_source']}**"
    )
    md.append("")
    md.append("## 💻 Environment")
    md.append(f"- Python: `{env['python_version'].splitlines()[0]}`")
    md.append(f"- Platform: `{env['platform']}`")
    md.append("")
    md.append("> Full metadata is also stored in the notebook metadata under `agg_metadata` "
              "and in the JSON file next to this notebook.")
    return "\n".join(md)


def build_todo_markdown(todo_items):
    """
    Build a TODO summary markdown cell.
    """
    md = []
    md.append("## ✅ Summary of Things That Need To Be Done (Auto-detected TODOs)")
    if not todo_items:
        md.append("")
        md.append("No `TODO` or `FIXME` comments were detected in the source notebook.")
        md.append("")
        md.append("You can add TODO comments like:")
        md.append("```python")
        md.append("# TODO: investigate missing values in MonthlyCharges")
        md.append("# FIXME: refactor this loop into a function")
        md.append("```")
        return "\n".join(md)

    md.append("")
    md.append(f"Detected **{len(todo_items)}** TODO/FIXME comment(s) in the source notebook:")
    md.append("")
    for item in todo_items:
        md.append(f"- {item}")
    md.append("")
    md.append("> Tip: As you refactor 01_EDA.ipynb into functions/modules, you can update or clear "
              "these TODOs and re-run this aggregation script to regenerate an updated TODO summary.")
    return "\n".join(md)


def build_aggregated_code_cell(group_key: str, grp: dict):
    """
    Build a single aggregated code cell for a section group by concatenating
    all original code cells for that group with clear BEGIN/END markers.
    """
    parts = []
    for i, cell_info in enumerate(grp["cells"], start=1):
        idx = cell_info["source_index"]
        parts.append(
            f"# ----- BEGIN original code cell {idx} "
            f"(#{i} in section {group_key}) -----"
        )
        parts.append(cell_info["source"])
        parts.append(f"# ----- END original code cell {idx} -----")
        parts.append("")  # spacer

    combined_source = "\n".join(parts).rstrip() + "\n"
    new_cell = nbformat.v4.new_code_cell(source=combined_source)
    new_cell.execution_count = None
    new_cell.outputs = []
    new_cell.metadata["agg_source_section_group"] = group_key
    new_cell.metadata["agg_original_cell_indices"] = [
        ci["source_index"] for ci in grp["cells"]
    ]
    return new_cell


# ────────────────────────────────────────────────────────────────────────────────
# MAIN
# ────────────────────────────────────────────────────────────────────────────────

def main():
    print(f"🔍 Loading source notebook: {NOTEBOOK_PATH}")
    nb_src = load_notebook(NOTEBOOK_PATH)

    print("📂 Ensuring output directory exists...")
    ensure_output_dir(OUTPUT_DIR)
    print(f"   → {OUTPUT_DIR}")

    print("🧬 Extracting code cells + section groups...")
    section_groups, code_stats, todo_items = extract_code_cells_and_groups(nb_src)

    print("🧾 Building metadata...")
    metadata = build_metadata(nb_src, code_stats, todo_items, section_groups)

    # Build new notebook
    print("🛠️ Building aggregated notebook object...")
    nb_new = new_notebook()

    # Carry over kernelspec/language_info if present
    for key in ("kernelspec", "language_info"):
        if key in nb_src.metadata:
            nb_new.metadata[key] = nb_src.metadata[key]

    # Attach the aggregation metadata
    nb_new.metadata["agg_metadata"] = metadata

    # Add metadata + TODO cells
    print("🧱 Adding metadata + TODO summary markdown cells...")
    nb_new.cells.append(new_markdown_cell(build_metadata_markdown(metadata)))
    nb_new.cells.append(new_markdown_cell(build_todo_markdown(todo_items)))

    # Add grouped section headers + aggregated code cells
    print("➕ Adding section headers and aggregated code cell(s)...")
    sorted_groups = sorted(
        section_groups.items(),
        key=lambda kv: kv[1]["first_source_index"],
    )

    for group_key, grp in sorted_groups:
        if group_key == "UNSECTIONED":
            header_title = "🧩 Unsectioned Code Cells"
            header_desc = (
                "Cells that appear before any numbered section header "
                "or in areas without a 1.x / 1.1-style heading."
            )
        else:
            header_title = f"📚 Section {group_key} — {grp['title']}"
            header_desc = "All original code cells for this section merged into one."

        header_md = []
        header_md.append(f"## {header_title}")
        header_md.append("")
        header_md.append(header_desc)
        header_md.append("")
        header_md.append(f"- Original code cells merged into this section: **{len(grp['cells'])}**")
        if grp["full_tags"]:
            tags_str = ", ".join(sorted(grp["full_tags"]))
            header_md.append(f"- Full section tags seen: `{tags_str}`")
        nb_new.cells.append(new_markdown_cell("\n".join(header_md)))

        # Single aggregated code cell for the section
        agg_cell = build_aggregated_code_cell(group_key, grp)
        nb_new.cells.append(agg_cell)

    # Write notebook
    print(f"💾 Writing aggregated notebook to: {OUTPUT_NOTEBOOK_PATH}")
    nbformat.write(nb_new, OUTPUT_NOTEBOOK_PATH)

    # Write metadata JSON
    print(f"💾 Writing metadata JSON to: {OUTPUT_METADATA_JSON}")
    with OUTPUT_METADATA_JSON.open("w", encoding="utf-8") as f:
        json.dump(metadata, f, indent=2)

    # Console summary
    agg_stats = metadata["aggregation_stats"]
    print("\n✅ Done.")
    print(f"   Aggregated notebook: {OUTPUT_NOTEBOOK_PATH}")
    print(f"   Metadata JSON:       {OUTPUT_METADATA_JSON}")
    print(f"   Non-empty code cells scraped from source: "
          f"{agg_stats['non_empty_code_cells_scraped_from_source']}")
    print(f"   Total cells created in destination: "
          f"{agg_stats['total_cells_created_in_destination']}")
    print(f"   → Code cells in destination (1 per section): "
          f"{agg_stats['code_cells_in_destination']}")
    print(f"   → Section groups: {agg_stats['section_group_count']}")


if __name__ == "__main__":
    main()

# AGGREGATE_PYTHON_CELLS_SCRIPT
"""
Aggregate all Python code cells from a Jupyter notebook into a single code cell
in a new notebook, skipping this script cell.

- Input:  SOURCE_NOTEBOOK
- Output: reports/AGG_ALL_{timestamp}.ipynb (one level up from the notebook dir)
"""

import nbformat
from pathlib import Path
from datetime import datetime

# ─────────────────────────────────────────
# 1. CONFIG: paths
# ─────────────────────────────────────────

SOURCE_NOTEBOOK = Path("/Users/b/DATA/PROJECTS/Telco/Level_3/notebooks/01_EDA.ipynb")

# destination: one level up from "notebooks/" → "reports/"
REPORTS_DIR = SOURCE_NOTEBOOK.parent.parent / "reports"
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

# marker used to identify this script cell so we can skip it
SCRIPT_MARKER = "AGGREGATE_PYTHON_CELLS_SCRIPT"

# ─────────────────────────────────────────
# 2. Load source notebook
# ─────────────────────────────────────────

nb = nbformat.read(SOURCE_NOTEBOOK, as_version=4)

included_cells = []
skipped_cells = 0
total_cells = len(nb.cells)

for idx, cell in enumerate(nb.cells):
    if cell.cell_type != "code":
        continue

    # Normalize source to a single string
    src = cell.source if isinstance(cell.source, str) else "".join(cell.source)

    # Skip the aggregator script cell (any cell containing the marker)
    if SCRIPT_MARKER in src:
        skipped_cells += 1
        continue

    included_cells.append((idx, src))

# ─────────────────────────────────────────
# 3. Build aggregated source (one big code cell)
# ─────────────────────────────────────────

aggregated_lines = []

for logical_idx, (cell_idx, src) in enumerate(included_cells, start=1):
    header = f"# --- Source code cell {cell_idx} (logical #{logical_idx}) ---\n"
    aggregated_lines.append(header)
    aggregated_lines.append(src.rstrip() + "\n\n")

agg_source = "".join(aggregated_lines).rstrip() + "\n"

# ─────────────────────────────────────────
# 4. Create new notebook with single code cell
# ─────────────────────────────────────────

timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
output_name = f"AGG_ALL_{timestamp}.ipynb"
OUTPUT_PATH = REPORTS_DIR / output_name

new_nb = nbformat.v4.new_notebook(
    metadata={
        "kernelspec": nb.metadata.get("kernelspec", {}),
        "language_info": nb.metadata.get("language_info", {}),
        "aggregated_from": str(SOURCE_NOTEBOOK),
        "aggregated_at_utc": timestamp,
    }
)

# Header markdown cell
new_nb.cells.append(
    nbformat.v4.new_markdown_cell(
        f"# Aggregated Python Cells\n\n"
        f"- Source: `{SOURCE_NOTEBOOK}`\n"
        f"- Generated: `{timestamp} UTC`\n"
        f"- Included code cells: `{len(included_cells)}`\n"
        f"- Skipped script cells (marker `{SCRIPT_MARKER}`): `{skipped_cells}`\n"
    )
)

# Single big code cell
new_nb.cells.append(
    nbformat.v4.new_code_cell(agg_source)
)

nbformat.write(new_nb, OUTPUT_PATH)

print("✅ Aggregation complete")
print(f"   Source notebook:       {SOURCE_NOTEBOOK}")
print(f"   Total cells scanned:   {total_cells}")
print(f"   Code cells included:   {len(included_cells)}")
print(f"   Script cells skipped:  {skipped_cells}")
print(f"   Output notebook:       {OUTPUT_PATH}")


In [ ]:
import json
from pathlib import Path
from datetime import datetime
import platform
import sys

# 1. SET THE PATH TO YOUR SOURCE NOTEBOOK
# IMPORTANT: Change this to the actual path of the notebook you want to aggregate.
notebook_path = Path("/Users/b/DATA/PROJECTS/Telco/Level_3/notebooks/01_EDA.ipynb") 

# --- Setup Output Paths ---
start_time_utc = datetime.utcnow()
today_str = start_time_utc.strftime("%Y%m%d%H%M")
output_filename = f"AGG_ALL_{today_str}.ipynb"

# Define output directory: one level up from notebook's folder, then into 'reports'
OUTPUT_DIR = notebook_path.parent.parent / "reports"
OUTPUT_PATH = OUTPUT_DIR / output_filename

# Ensure the output directory exists
try:
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
except Exception as e:
    print(f"❌ ERROR: Could not create output directory {OUTPUT_DIR}: {e}")
    raise

print(f"Attempting to read source notebook: {notebook_path.resolve()}")

# --- Core Extraction Logic ---
new_notebook_cells = []
source_kernel_spec = {
    "display_name": "Python 3 (ipykernel)", 
    "language": "python", 
    "name": "python3"
}
cells_extracted_count = 0

if not notebook_path.exists():
    print(f"❌ ERROR: Source file not found at {notebook_path.resolve()}")
else:
    try:
        with open(notebook_path, 'r', encoding='utf-8') as f:
            notebook_json = json.load(f)

        # Try to capture the source kernel spec for the new notebook
        if 'kernelspec' in notebook_json.get('metadata', {}):
            source_kernel_spec = notebook_json['metadata']['kernelspec']
            
        if 'cells' not in notebook_json:
            print("❌ ERROR: Invalid notebook format. Missing 'cells' key.")
        else:
            
            # --- 2. Iterate and Extract Code Cells (WITHOUT added comments) ---
            for cell in notebook_json['cells']:
                if cell.get('cell_type') == 'code':
                    source_lines = cell.get('source', [])
                    cell_code_list = []
                    
                    if isinstance(source_lines, list):
                        # Join and strip to check if it's empty
                        if not "".join(source_lines).strip():
                            continue
                        cell_code_list = source_lines
                        
                    elif isinstance(source_lines, str):
                        if not source_lines.strip():
                            continue
                        cell_code_list = [source_lines]

                    # Create the JSON structure for a clean code cell
                    new_notebook_cells.append({
                        "cell_type": "code",
                        "execution_count": None, 
                        "metadata": {},
                        "outputs": [],
                        "source": cell_code_list
                    })
                    cells_extracted_count += 1

            
            # --- 3. Create Header Cell with Metadata ---
            # This is done *after* extraction to get the correct cell count.
            markdown_source = [
                f"# 📊 Aggregated Code Report: {notebook_path.name}\n\n",
                "This notebook contains a cleaned, sequential aggregation of all ",
                f"**{cells_extracted_count}** non-empty Python code cells ",
                "extracted from the source notebook.\n\n",
                "---",
                f"\n**Report Creation Time (UTC):** `{start_time_utc.strftime('%Y-%m-%d %H:%M:%S')}`",
                f"\n**Source Path:** `{notebook_path.resolve()}`",
                f"\n**Python Version:** `{platform.python_version()}`",
                f"\n**Kernel:** `{source_kernel_spec.get('display_name', 'Unknown')}`",
            ]
            header_cell = {
                "cell_type": "markdown",
                "metadata": {},
                "source": markdown_source
            }
            
            # Insert the header cell at the very beginning
            new_notebook_cells.insert(0, header_cell)
            
            
            # --- 4. Assemble and Save the New Notebook JSON ---
            if cells_extracted_count > 0:
                notebook_content = {
                    "cells": new_notebook_cells,
                    "metadata": {
                        "kernelspec": source_kernel_spec,
                        "language_info": {"name": "python"},
                    },
                    "nbformat": 4, 
                    "nbformat_minor": 5
                }
                
                with open(OUTPUT_PATH, 'w', encoding='utf-8') as outfile:
                    json.dump(notebook_content, outfile, indent=4)
                
                print(f"\n✅ Success! Aggregated {cells_extracted_count} code cells.")
                print(f"💾 Notebook saved to: {OUTPUT_PATH.resolve()}")

            else:
                print("⚠️ No code cells found to aggregate.")

    except json.JSONDecodeError:
        print(f"❌ ERROR: Could not decode JSON from {notebook_path.name}.")
    except Exception as e:
        print(f"❌ An unexpected error occurred: {e}")
import sys
from pathlib import Path
from datetime import datetime

import nbformat
from nbformat.v4 import new_notebook, new_markdown_cell, new_code_cell

# ─────────────────────────────────────────────────────────
# 1. Locate the current notebook
# ─────────────────────────────────────────────────────────
# This assumes you're running in an environment (e.g. VS Code / Windsurf)
# where sys.argv[0] is the path to the .ipynb file.
nb_candidate = Path(sys.argv[0])
nb_path = Path('/Users/b/DATA/PROJECTS/Telco/Level_3/notebooks/01_EDA.ipynb').resolve()

# if nb_candidate.suffix == ".ipynb" and nb_candidate.exists():
#     nb_path = nb_candidate.resolve()
# else:
#     raise RuntimeError(
#         "Could not automatically determine notebook path.\n"
#         "Set nb_path manually, e.g.:\n"
#         "  nb_path = Path('01_EDA.ipynb').resolve()"
#     )

print(f"📄 Source notebook: {nb_path}")

# ─────────────────────────────────────────────────────────
# 2. Read the notebook & collect all code (Python) cells
# ─────────────────────────────────────────────────────────
nb = nbformat.read(nb_path, as_version=4)

code_cells = [cell for cell in nb.cells if cell.get("cell_type") == "code"]
n_code_cells = len(code_cells)
print(f"🔢 Found {n_code_cells} code cells to aggregate.")

# ─────────────────────────────────────────────────────────
# 3. Build header cell with metadata (markdown only)
# ─────────────────────────────────────────────────────────
run_ts = datetime.utcnow().isoformat(timespec="seconds") + "Z"
cwd = Path.cwd()
py_ver = sys.version.split()[0]

header_md = f"""# Aggregated Python Cells Report

- **Source notebook:** `{nb_path.name}`
- **Created at (UTC):** `{run_ts}`
- **Number of code cells:** `{n_code_cells}`
- **Working directory when created:** `{cwd}`
- **Python version:** `{py_ver}`

> This notebook was auto-generated by an inline aggregation script.
> All code cells below are copied verbatim from the source notebook.
"""

header_cell = new_markdown_cell(header_md)

# ─────────────────────────────────────────────────────────
# 4. Construct the new notebook (header + raw code cells)
#    NOTE: we do *not* modify any code cell contents.
# ─────────────────────────────────────────────────────────
out_cells = [header_cell]

for cell in code_cells:
    # Copy source exactly; no comments/prints are added to the code.
    out_cells.append(
        new_code_cell(
            source=cell.get("source", ""),
            metadata=cell.get("metadata", {})
        )
    )

out_nb = new_notebook(
    cells=out_cells,
    metadata={
        "aggregated_from": str(nb_path),
        "created_utc": run_ts,
        "n_code_cells": n_code_cells,
        "tool": "agg_all_inline_v1"
    }
)

# ─────────────────────────────────────────────────────────
# 5. Save as AGG_ALL_{date}.ipynb in ../reports
# ─────────────────────────────────────────────────────────
# "reports folder a level up" relative to the notebook file:
reports_dir = nb_path.parent.parent / "reports"
reports_dir.mkdir(parents=True, exist_ok=True)

ts_str = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
out_path = reports_dir / f"AGG_ALL_{ts_str}.ipynb"

nbformat.write(out_nb, out_path)

print(f"✅ Aggregated notebook written → {out_path}")

import json
from pathlib import Path
from datetime import datetime
import platform
import sys

# ============================================================================
# 0. CONFIG: where your notebooks live, and which ones belong to each section
# ============================================================================

# Base notebooks directory
BASE_NOTEBOOK_DIR = Path("/Users/b/DATA/PROJECTS/Telco/Level_3/notebooks")

# Map logical sections → filename patterns (glob)
# Adjust patterns to match your project naming.
SECTION_PATTERNS = {
    "Section 1": ["01_*.ipynb", "1_*.ipynb"],
    "Section 2": ["02_*.ipynb", "2_*.ipynb"],
    "Section 3": ["03_*.ipynb", "3_*.ipynb"],
}

# ----------------------------------------------------------------------------
# Helper: expand glob patterns to concrete notebook paths
# ----------------------------------------------------------------------------
def _expand_patterns(base_dir: Path, patterns):
    paths = []
    for pat in patterns:
        paths.extend(sorted(base_dir.glob(pat)))
    # De-duplicate while preserving order
    seen = set()
    uniq = []
    for p in paths:
        if p not in seen:
            uniq.append(p)
            seen.add(p)
    return uniq

# ============================================================================
# 1. Resolve notebooks per section + ensure all source notebooks are included
# ============================================================================

if not BASE_NOTEBOOK_DIR.exists():
    raise FileNotFoundError(f"Notebook directory not found: {BASE_NOTEBOOK_DIR}")

sections_notebooks = {}
all_assigned = set()

for section_label, patterns in SECTION_PATTERNS.items():
    nb_paths = _expand_patterns(BASE_NOTEBOOK_DIR, patterns)
    sections_notebooks[section_label] = nb_paths
    all_assigned.update(nb_paths)

# All "source" notebooks in this dir (excluding previous AGG_ALL outputs)
all_notebooks = sorted(
    p for p in BASE_NOTEBOOK_DIR.glob("*.ipynb")
    if not p.name.startswith("AGG_ALL_")
)

unassigned = [p for p in all_notebooks if p not in all_assigned]

if unassigned:
    print("⚠️ The following notebooks did not match any SECTION_PATTERNS "
          "and will be auto-assigned to Section 3:")
    for p in unassigned:
        print(f"   - {p.name}")
    # Ensure Section 3 exists
    sections_notebooks.setdefault("Section 3", [])
    sections_notebooks["Section 3"].extend(unassigned)

# ============================================================================
# 2. Setup output paths
# ============================================================================

start_time_utc = datetime.utcnow()
today_str = start_time_utc.strftime("%Y%m%d%H%M")
output_filename = f"AGG_ALL_{today_str}.ipynb"

OUTPUT_DIR = BASE_NOTEBOOK_DIR.parent / "reports"
OUTPUT_PATH = OUTPUT_DIR / output_filename

try:
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
except Exception as e:
    print(f"❌ ERROR: Could not create output directory {OUTPUT_DIR}: {e}")
    raise

print(f"📂 Base notebook directory: {BASE_NOTEBOOK_DIR.resolve()}")
print(f"💾 Aggregated notebook will be saved to: {OUTPUT_PATH.resolve()}")

# ============================================================================
# 3. Aggregate code cells, grouped by Section 1 / Section 2 / Section 3
# ============================================================================

new_notebook_cells = []
source_kernel_spec = {
    "display_name": "Python 3 (ipykernel)",
    "language": "python",
    "name": "python3",
}
got_kernel_spec = False

total_code_cells = 0
section_code_counts = {sec: 0 for sec in sections_notebooks.keys()}
per_notebook_counts = {}

for section_label in ["Section 1", "Section 2", "Section 3"]:
    nb_paths = sections_notebooks.get(section_label, [])
    if not nb_paths:
        continue

    # Section header (markdown only)
    new_notebook_cells.append({
        "cell_type": "markdown",
        "metadata": {},
        "source": [f"# {section_label}\n\n"]
    })

    for nb_path in nb_paths:
        nb_path = nb_path.resolve()
        if not nb_path.exists():
            print(f"❌ ERROR: Notebook not found: {nb_path}")
            continue

        print(f"🔍 Processing {section_label}: {nb_path.name}")

        try:
            with open(nb_path, "r", encoding="utf-8") as f:
                notebook_json = json.load(f)
        except json.JSONDecodeError:
            print(f"❌ ERROR: Could not decode JSON from {nb_path.name}. Skipping.")
            continue
        except Exception as e:
            print(f"❌ ERROR reading {nb_path.name}: {e}")
            continue

        # Capture kernelspec once (from first valid source notebook)
        if not got_kernel_spec and "kernelspec" in notebook_json.get("metadata", {}):
            source_kernel_spec = notebook_json["metadata"]["kernelspec"]
            got_kernel_spec = True

        if "cells" not in notebook_json:
            print(f"❌ ERROR: Invalid notebook format (no 'cells') in {nb_path.name}.")
            continue

        nb_code_count = 0

        # Per-notebook header (markdown only)
        new_notebook_cells.append({
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                f"## {nb_path.name}\n\n",
                f"_Source: `{nb_path}`_\n"
            ],
        })

        # Extract code cells (Python) without modifying their contents
        for cell in notebook_json["cells"]:
            if cell.get("cell_type") != "code":
                continue

            source_lines = cell.get("source", [])
            cell_code_list = []

            if isinstance(source_lines, list):
                if not "".join(source_lines).strip():
                    continue  # skip truly empty code cells
                cell_code_list = source_lines
            elif isinstance(source_lines, str):
                if not source_lines.strip():
                    continue
                cell_code_list = [source_lines]
            else:
                # Unknown source type, skip
                continue

            new_notebook_cells.append({
                "cell_type": "code",
                "execution_count": None,
                "metadata": {},
                "outputs": [],
                "source": cell_code_list,  # ← exact code, no added text
            })
            nb_code_count += 1
            total_code_cells += 1
            section_code_counts[section_label] += 1

        per_notebook_counts[str(nb_path)] = nb_code_count

# ============================================================================
# 4. Build top-level header cell with metadata (after counts are known)
# ============================================================================

markdown_source = []

markdown_source.append("# 📊 Aggregated Code Report: Sections 1–3\n\n")
markdown_source.append(
    "This notebook contains a cleaned, sequential aggregation of all "
    f"**{total_code_cells}** non-empty Python code cells grouped into "
    "**Section 1**, **Section 2**, and **Section 3**.\n\n"
)
markdown_source.append("---")

markdown_source.append(
    f"\n**Report Creation Time (UTC):** "
    f"`{start_time_utc.strftime('%Y-%m-%d %H:%M:%S')}`"
)
markdown_source.append(
    f"\n**Notebook Base Directory:** `{BASE_NOTEBOOK_DIR.resolve()}`"
)
markdown_source.append(
    f"\n**Python Version:** `{platform.python_version()}`"
)
markdown_source.append(
    f"\n**Kernel (source):** "
    f"`{source_kernel_spec.get('display_name', 'Unknown')}`"
)

markdown_source.append("\n\n**Section breakdown (code cells):**\n")
for sec in ["Section 1", "Section 2", "Section 3"]:
    if sec in section_code_counts:
        markdown_source.append(
            f"- {sec}: {section_code_counts[sec]} code cells\n"
        )

markdown_source.append("\n**Included notebooks:**\n")
for sec in ["Section 1", "Section 2", "Section 3"]:
    nb_paths = sections_notebooks.get(sec, [])
    if not nb_paths:
        continue
    markdown_source.append(f"- {sec}:\n")
    for nb_path in nb_paths:
        nb_path_str = str(nb_path.resolve())
        count = per_notebook_counts.get(nb_path_str, 0)
        markdown_source.append(
            f"  - `{Path(nb_path_str).name}` ({count} code cells)\n"
        )

header_cell = {
    "cell_type": "markdown",
    "metadata": {},
    "source": markdown_source,
}

# Insert header at very beginning
new_notebook_cells.insert(0, header_cell)

# ============================================================================
# 5. Assemble and save the new aggregated notebook
# ============================================================================

if total_code_cells == 0:
    print("⚠️ No code cells found to aggregate. Notebook will not be written.")
else:
    notebook_content = {
        "cells": new_notebook_cells,
        "metadata": {
            "kernelspec": source_kernel_spec,
            "language_info": {"name": "python"},
            "aggregated_sections": ["Section 1", "Section 2", "Section 3"],
            "aggregated_from_dir": str(BASE_NOTEBOOK_DIR.resolve()),
            "created_utc": start_time_utc.strftime('%Y-%m-%dT%H:%M:%SZ'),
            "total_code_cells": total_code_cells,
            "section_code_counts": section_code_counts,
            "tool": "agg_all_sections_v1",
        },
        "nbformat": 4,
        "nbformat_minor": 5,
    }

    with open(OUTPUT_PATH, "w", encoding="utf-8") as outfile:
        json.dump(notebook_content, outfile, indent=4)

    print(f"\n✅ Success! Aggregated {total_code_cells} code cells in total.")
    print(f"💾 Notebook saved to: {OUTPUT_PATH.resolve()}")

import json
from pathlib import Path
from datetime import datetime
import platform

# ─────────────────────────────────────────────────────
# 1. SET THE PATH TO YOUR SOURCE NOTEBOOK (fixed)
# ─────────────────────────────────────────────────────
notebook_path = Path("/Users/b/DATA/PROJECTS/Telco/Level_3/notebooks/01_EDA.ipynb")

# --- Setup Output Paths ---
start_time_utc = datetime.utcnow()
today_str = start_time_utc.strftime("%Y%m%d%H%M")
output_filename = f"AGG_ALL_{today_str}.ipynb"

# Output: one level up from notebook folder, in 'reports'
OUTPUT_DIR = notebook_path.parent.parent / "reports"
OUTPUT_PATH = OUTPUT_DIR / output_filename

# Ensure the output directory exists
try:
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
except Exception as e:
    print(f"❌ ERROR: Could not create output directory {OUTPUT_DIR}: {e}")
    raise

print(f"Attempting to read source notebook: {notebook_path.resolve()}")

# ─────────────────────────────────────────────────────
# 2. Core Extraction Logic (Section 1 / 2 / 3)
# ─────────────────────────────────────────────────────

# Buckets for sectioned code cells
section_cells = {
    "Section 1": [],
    "Section 2": [],
    "Section 3": [],
}

source_kernel_spec = {
    "display_name": "Python 3 (ipykernel)",
    "language": "python",
    "name": "python3",
}
cells_extracted_count = 0

def _detect_section_from_markdown(md_source, current_section):
    """
    Look at markdown text and update section if it contains 'Section 1/2/3'.
    Returns the new current_section.
    """
    if isinstance(md_source, list):
        text = "".join(md_source)
    else:
        text = str(md_source)
    lower = text.lower()

    if "section 3" in lower:
        return "Section 3"
    if "section 2" in lower:
        return "Section 2"
    if "section 1" in lower:
        return "Section 1"
    return current_section

if not notebook_path.exists():
    print(f"❌ ERROR: Source file not found at {notebook_path.resolve()}")
else:
    try:
        with open(notebook_path, "r", encoding="utf-8") as f:
            notebook_json = json.load(f)

        # Try to capture the source kernel spec for the new notebook
        if "kernelspec" in notebook_json.get("metadata", {}):
            source_kernel_spec = notebook_json["metadata"]["kernelspec"]

        if "cells" not in notebook_json:
            print("❌ ERROR: Invalid notebook format. Missing 'cells' key.")
        else:
            # Default: assume we're in Section 1 until a header says otherwise
            current_section = "Section 1"

            # Walk through all cells in order
            for cell in notebook_json["cells"]:
                cell_type = cell.get("cell_type")

                if cell_type == "markdown":
                    # Update current_section if markdown mentions a Section heading
                    current_section = _detect_section_from_markdown(
                        cell.get("source", ""), current_section
                    )

                elif cell_type == "code":
                    # Extract code cell into current section bucket
                    source_lines = cell.get("source", [])
                    cell_code_list = []

                    if isinstance(source_lines, list):
                        if not "".join(source_lines).strip():
                            continue  # skip truly empty cells
                        cell_code_list = source_lines
                    elif isinstance(source_lines, str):
                        if not source_lines.strip():
                            continue
                        cell_code_list = [source_lines]
                    else:
                        continue  # unknown format, skip

                    # Create a clean code cell (NO added content to 'source')
                    code_cell = {
                        "cell_type": "code",
                        "execution_count": None,
                        "metadata": {},
                        "outputs": [],
                        "source": cell_code_list,
                    }

                    # Ensure section exists, then append
                    section_cells.setdefault(current_section, []).append(code_cell)
                    cells_extracted_count += 1

            # ─────────────────────────────────────────────
            # 3. Build the new notebook cell list
            # ─────────────────────────────────────────────
            new_notebook_cells = []

            # Header markdown (overall metadata)
            header_md = [
                "# 📊 Aggregated Code Report: 01_EDA Sections 1–3\n\n",
                "This notebook contains a cleaned, sequential aggregation of all ",
                f"**{cells_extracted_count}** non-empty Python code cells ",
                "from the source notebook, grouped into **Section 1**, **Section 2**, and **Section 3** ",
                "based on markdown headings that mention those sections.\n\n",
                "---",
                f"\n**Report Creation Time (UTC):** `{start_time_utc.strftime('%Y-%m-%d %H:%M:%S')}`",
                f"\n**Source Path:** `{notebook_path.resolve()}`",
                f"\n**Python Version:** `{platform.python_version()}`",
                f"\n**Kernel:** `{source_kernel_spec.get('display_name', 'Unknown')}`",
                "\n\n**Section breakdown (code cells):**\n",
            ]

            for sec_name in ["Section 1", "Section 2", "Section 3"]:
                header_md.append(
                    f"- {sec_name}: {len(section_cells.get(sec_name, []))} code cells\n"
                )

            header_cell = {
                "cell_type": "markdown",
                "metadata": {},
                "source": header_md,
            }

            new_notebook_cells.append(header_cell)

            # Add per-section markdown headers + code cells
            for sec_name in ["Section 1", "Section 2", "Section 3"]:
                cells_in_section = section_cells.get(sec_name, [])
                if not cells_in_section:
                    continue  # skip empty sections

                # Section header
                new_notebook_cells.append(
                    {
                        "cell_type": "markdown",
                        "metadata": {},
                        "source": [f"# {sec_name}\n\n"],
                    }
                )

                # Code cells (verbatim source)
                new_notebook_cells.extend(cells_in_section)

            # ─────────────────────────────────────────────
            # 4. Assemble and save the new notebook JSON
            # ─────────────────────────────────────────────
            if cells_extracted_count > 0:
                notebook_content = {
                    "cells": new_notebook_cells,
                    "metadata": {
                        "kernelspec": source_kernel_spec,
                        "language_info": {"name": "python"},
                        "aggregated_from": str(notebook_path.resolve()),
                        "created_utc": start_time_utc.strftime("%Y-%m-%dT%H:%M:%SZ"),
                        "total_code_cells": cells_extracted_count,
                        "section_code_counts": {
                            sec: len(cells)
                            for sec, cells in section_cells.items()
                        },
                        "tool": "agg_single_notebook_sections_v1",
                    },
                    "nbformat": 4,
                    "nbformat_minor": 5,
                }

                with open(OUTPUT_PATH, "w", encoding="utf-8") as outfile:
                    json.dump(notebook_content, outfile, indent=4)

                print(f"\n✅ Success! Aggregated {cells_extracted_count} code cells.")
                print(f"💾 Notebook saved to: {OUTPUT_PATH.resolve()}")
            else:
                print("⚠️ No code cells found to aggregate.")

    except json.JSONDecodeError:
        print(f"❌ ERROR: Could not decode JSON from {notebook_path.name}.")
    except Exception as e:
        print(f"❌ An unexpected error occurred: {e}")


In [ ]:
# 📜 AGGREGATE Python cells THAT START WITH "# 1"

from pathlib import Path
import json

# --- Define your notebook path ---
NOTEBOOK_PATH = Path("01_EDA.ipynb")   # or Path(__file__).parent / "01_Environment.ipynb"
OUTPUT_PATH   = NOTEBOOK_PATH.with_name("aggregated_section1.py")

# --- Load notebook JSON ---
with open(NOTEBOOK_PATH, "r", encoding="utf-8") as f:
    nb = json.load(f)

print(f"📘 Loaded notebook → {NOTEBOOK_PATH.name}")
print("🔍 Searching for Python cells starting with '# 1' ...")

# --- Extract code cells whose *first line* starts with '# 1'
aggregated_blocks = []

for cell in nb["cells"]:
    if cell.get("cell_type") == "code":
        src = "".join(cell.get("source", []))
        first_line = src.strip().splitlines()[0] if src.strip() else ""
        if first_line.startswith("# 1"):
            aggregated_blocks.append(src.strip())

# --- Combine all matches ---
aggregated_script = "\n\n# ════════════════════════════════════════════════════════════════════════════════\n\n".join(aggregated_blocks)

# --- Write output (if any) ---
if aggregated_blocks:
    OUTPUT_PATH.write_text(aggregated_script, encoding="utf-8")
    print(f"✅ Aggregated {len(aggregated_blocks)} cells → {OUTPUT_PATH}")
else:
    print("⚠️ No code cells starting with '# 1' were found.")

# Optional: Display preview
print("\n--- Aggregated Script Preview ---")
print("\n".join(aggregated_script.splitlines()[:20]))

# 📜 AGGREGATE Section 2.0 python

from pathlib import Path
import json

# --- Define your notebook path ---
NOTEBOOK_PATH = Path("01_EDA.ipynb")   # or Path(__file__).parent / "01_Environment.ipynb"
OUTPUT_PATH   = NOTEBOOK_PATH.with_name("aggregated_section2_0.py")

# --- Load notebook JSON ---
with open(NOTEBOOK_PATH, "r", encoding="utf-8") as f:
    nb = json.load(f)

print(f"📘 Loaded notebook → {NOTEBOOK_PATH.name}")
print("🔍 Searching for Python cells starting with '# 2.0' ...")

# --- Extract code cells whose *first line* starts with '# 2.0'
aggregated_blocks = []

for cell in nb["cells"]:
    if cell.get("cell_type") == "code":
        src = "".join(cell.get("source", []))
        first_line = src.strip().splitlines()[0] if src.strip() else ""
        if first_line.startswith("# 2.0"):
            aggregated_blocks.append(src.strip())

# --- Combine all matches ---
aggregated_script = "\n\n# ════════════════════════════════════════════════════════════════════════════════\n\n".join(aggregated_blocks)

# --- Write output (if any) ---
if aggregated_blocks:
    OUTPUT_PATH.write_text(aggregated_script, encoding="utf-8")
    print(f"✅ Aggregated {len(aggregated_blocks)} cells → {OUTPUT_PATH}")
else:
    print("⚠️ No code cells starting with '# 2.0' were found.")

# Optional: Display preview
print("\n--- Aggregated Script Preview ---")
print("\n".join(aggregated_script.splitlines()[:20]))

# Aggregating Section 2 Code into .py
import json, io, os
from pathlib import Path
from IPython.display import Markdown, display

# ---------------------------------------------------------------------
# 1️⃣ Try to infer current notebook name (works in most Jupyter setups)
# ---------------------------------------------------------------------
try:
    from IPython import get_ipython
    ip = get_ipython()
    if ip is not None and hasattr(ip, 'kernel'):
        connection_file = Path(ip.kernel.connection_file)
        notebook_name = next(
            (p.name for p in Path('.').glob('*.ipynb') if str(p.stat().st_ino) in str(connection_file)),
            None
        )
    else:
        notebook_name = None
except Exception:
    notebook_name = None

# fallback manual name if automatic detection fails
if not notebook_name:
    notebook_name = "01_EDA.ipynb"

NOTEBOOK_PATH = Path(notebook_name)

# ---------------------------------------------------------------------
# 2️⃣ Load the notebook JSON
# ---------------------------------------------------------------------
if not NOTEBOOK_PATH.exists():
    raise FileNotFoundError(f"Notebook not found: {NOTEBOOK_PATH.resolve()}")

with open(NOTEBOOK_PATH, "r", encoding="utf-8") as f:
    nb = json.load(f)

# ---------------------------------------------------------------------
# 3️⃣ Collect code cells whose first non-empty line starts with "# 2"
# ---------------------------------------------------------------------
section_code = []
for cell in nb.get("cells", []):
    if cell.get("cell_type") == "code":
        src = "".join(cell.get("source", []))
        if not src.strip():
            continue
        first_line = next((ln.strip() for ln in src.splitlines() if ln.strip()), "")
        if first_line.startswith("# 2"):
            section_code.append(src.rstrip())

# ---------------------------------------------------------------------
# 4️⃣ Aggregate and render inline
# ---------------------------------------------------------------------
# if section_code:
#     combined = "\n\n".join(section_code)
#     md = f"```python\n{combined}\n```"
#     display(Markdown(f"### 🧩 Aggregated Section 2 Code ({len(section_code)} cells)\n\n" + md))
# else:
#     display(Markdown("> ⚠️ **No code cells starting with `# 2` found in this notebook.**"))

Path("section2_aggregate.py").write_text(combined, encoding="utf-8")
print("✅ Saved section2_aggregate.py")

from pathlib import Path
import json

# --- Settings ---
notebook_path = Path("01_EDA.ipynb")  # change as needed

# --- Load notebook as JSON ---
nb = json.loads(notebook_path.read_text(encoding="utf-8"))

# --- Find the first code cell and its first comment line ---
first_comment = None
for cell in nb.get("cells", []):
    if cell.get("cell_type") == "code":
        for line in cell.get("source", []):
            line = line.strip()
            if line.startswith("#"):
                first_comment = line
                break
        if first_comment:
            break

# --- Output ---
if first_comment:
    print(f"✅ First section header found:\n{first_comment}")
else:
    print("⚠️ No comment header found in any code cell.")

# MD AGG First line headers 
from pathlib import Path
import json

# --- Settings ---
notebook_path = Path("01_EDA.ipynb")  # adjust to your notebook name

# --- Load notebook as JSON ---
nb = json.loads(notebook_path.read_text(encoding="utf-8"))

# --- Collect top-of-cell comment headers ---
headers = []

for cell in nb.get("cells", []):
    if cell.get("cell_type") == "code":
        src = cell.get("source", [])
        if not src:
            continue
        # Find the first non-empty line
        for line in src:
            stripped = line.strip()
            if stripped:  # skip blank lines
                if stripped.startswith("#"):
                    headers.append(stripped)
                break  # stop after the first meaningful line in this cell

# --- Output results ---
if headers:
    print(f"✅ Found {len(headers)} section headers:\n")
    for i, h in enumerate(headers, 1):
        print(f"{i:02}. {h}")
else:
    print("⚠️ No top-of-cell comment headers found.")

# agg_md.py
import json
from pathlib import Path
from datetime import datetime

# --- Locate Level_3 base robustly ---
cwd = Path.cwd()
level3 = next((p for p in [cwd, *cwd.parents] if p.name == "Level_3"), None)

if level3 is None:
    # Fallback to your explicit absolute path
    level3 = Path("/Users/b/DATA/PROJECTS/Telco/Level_3")

# --- Paths ---
input_path  = level3 / "notebooks" / "01_EDA.ipynb"
report_dir  = level3 / "resources" / "reports"
report_dir.mkdir(parents=True, exist_ok=True)

timestamp   = datetime.now().strftime("%m%d_%Y_%H%M%S")
output_path = report_dir / f"{timestamp}_Aggggggggg_01_EDA_Telco_Markdown.md"

# --- Guard: make sure the notebook exists where we think it does ---
if not input_path.exists():
    raise FileNotFoundError(f"Could not find notebook at: {input_path}\n"
                            f"Current working dir: {cwd}")

# --- Extract markdown cells ---
nb = json.loads(input_path.read_text(encoding="utf-8"))
md_cells = ["".join(c["source"]).strip() for c in nb["cells"] if c.get("cell_type") == "markdown"]

# --- Write to output ---
combined = "\n\n---\n\n".join(md_cells)
output_path.write_text(combined, encoding="utf-8")

print(f"✅ Wrote {len(md_cells)} markdown cells to {output_path}")

# agg_code.py :1
from nbformat import read
from pathlib import Path
from datetime import datetime

nb = read("01_EDA.ipynb", as_version=4)
code_cells = [cell.source for cell in nb.cells if cell.cell_type == "code"]
Path(f"aggregated_code_{datetime.now().strftime('%Y_%m%d_%H:%M')}.py").write_text("\n\n".join(code_cells))
print("✅ Extracted code written to aggregated_code_{datetime.now().strftime('%Y_%m%d_%H:%M')}.py")
#agg_code.py
from nbformat import read
from pathlib import Path
from datetime import datetime

# Locate the notebook and compute the Level_3 root from it
nb_path = Path("01_EDA.ipynb").resolve()
if not nb_path.exists():
    raise FileNotFoundError(f"Notebook not found at: {nb_path}")

level3_root = nb_path.parent.parent  # from .../notebooks -> .../Level_3

# Output directory: Level_3/resources/reports/script_agg_report
out_dir = level3_root / "resources" / "reports" / "script_agg"
out_dir.mkdir(parents=True, exist_ok=True)

# Read notebook and aggregate code cells
nb = read(str(nb_path), as_version=4)
code_cells = [cell.source for cell in nb.cells if getattr(cell, "cell_type", "") == "code"]

# Write output file with timestamp
timestamp = datetime.now().strftime("%Y_%m%d_%H%M")
out_path = out_dir / f"agg_{timestamp}.py"
out_path.write_text("\n\n".join(code_cells), encoding="utf-8")

print(f"✅ Extracted code written to {out_path}")

# agg all py cell/ first line
from pathlib import Path
import json

# --- Settings ---
notebook_path = Path("01_EDA.ipynb")  # path to your notebook

# --- Load the notebook JSON ---
nb = json.loads(notebook_path.read_text(encoding="utf-8"))

# --- Extract first line of every code cell ---
first_lines = []

for i, cell in enumerate(nb.get("cells", []), start=1):
    if cell.get("cell_type") == "code":
        src = cell.get("source", [])
        # Skip empty cells
        if not src:
            continue
        # Find the first non-empty line
        for line in src:
            stripped = line.strip()
            if stripped:
                first_lines.append((i, stripped))
                break

# --- Output results ---
if first_lines:
    print(f"✅ Found {len(first_lines)} code cells with content:\n")
    for idx, line in first_lines:
        print(f"{idx:02}. {line}")
else:
    print("⚠️ No code cells with content found.")


---

In [ ]:
# INLINE v9: Interactive notebook selection + interactive output directory
# - Finds 01_EDA.ipynb in repo
# - Lets you CLICK which notebook to use
# - Lets you CLICK where to save output
# - Aggregates a numeric section range into a new .ipynb

import json, re, time
from pathlib import Path

# ─────────────────────────────────────────────
# CONFIG — EDIT ONLY THESE
# ─────────────────────────────────────────────
RANGE_START_LABEL = "2.5"
RANGE_END_LABEL   = "7.2"

SOURCE_NOTEBOOK_NAME = "02_DQ_IF.ipynb"
MAX_DEPTH = 4
# ─────────────────────────────────────────────


# ─────────────────────────────────────────────
# Repo discovery + basic search
# ─────────────────────────────────────────────

def find_repo_root(start: Path) -> Path:
    """Walk upward from start to find a folder containing .git."""
    current = start.resolve()
    for parent in [current] + list(current.parents):
        if (parent / ".git").is_dir():
            return parent
    return current


def find_notebooks(repo_root: Path, name: str):
    """Return all matching paths inside the repo."""
    return list(repo_root.rglob(name))


def extract_level_num(path: Path) -> int:
    """Extract numeric Level_# from path; non-matches get huge number."""
    m = re.search(r"Level_(\d+)", str(path))
    return int(m.group(1)) if m else 999999


# ─────────────────────────────────────────────
# INTERACTIVE NOTEBOOK CHOOSER
# ─────────────────────────────────────────────

def format_notebook_label(path: Path, repo_root: Path) -> str:
    """Pretty label like 'Level_3  → notebooks/01_EDA.ipynb'."""
    s = str(path)
    level_match = re.search(r"(Level_\d+)", s)
    level = level_match.group(1) if level_match else "NoLevel"
    try:
        rel = path.relative_to(repo_root)
    except ValueError:
        rel = path
    return f"{level:<8} → {rel}"


def choose_notebook_interactive(matches, repo_root: Path) -> Path:
    """
    Interactive notebook selector:
    - Dropdown listing matches (sorted by Level_#)
    - 'Browse notebooks…' to pick any .ipynb (if ipyfilechooser installed)
    - 'Confirm' button to finalize selection
    """
    import ipywidgets as widgets
    from IPython.display import display

    # sort matches by numeric level
    matches = sorted(matches, key=extract_level_num)

    # map label -> path
    options = {format_notebook_label(m, repo_root): m for m in matches}

    dropdown = widgets.Dropdown(
        options=options,
        description='Notebook:',
        layout=widgets.Layout(width='100%')
    )

    browse_button = widgets.Button(description='Browse notebooks…')
    confirm_button = widgets.Button(description='Confirm', button_style='success')
    path_label = widgets.Label()
    out = widgets.Output()

    selected_nb = {"path": list(options.values())[0]}
    done_flag = {"done": False}

    def update_label():
        path_label.value = f"Selected notebook: {selected_nb['path']}"

    def on_dropdown_change(change):
        if change["name"] == "value":
            selected_nb["path"] = change["new"]
            update_label()

    dropdown.observe(on_dropdown_change)

    def on_browse_click(b):
        out.clear_output()
        with out:
            try:
                # Optional: notebook navigator (pip install ipyfilechooser)
                from ipyfilechooser import FileChooser

                fc = FileChooser(str(repo_root))
                fc.title = 'Pick a notebook (.ipynb)'
                fc.use_dir_icons = True
                fc.filter_pattern = '*.ipynb'

                def on_done(chooser):
                    if chooser.selected is None:
                        return
                    p = Path(chooser.selected)
                    selected_nb["path"] = p
                    # Add to dropdown options dynamically
                    label = format_notebook_label(p, repo_root)
                    options[label] = p
                    dropdown.options = options
                    dropdown.value = p
                    update_label()

                fc.register_callback(on_done)
                display(fc)

            except ImportError:
                print("ipyfilechooser not installed.")
                print("Run `pip install ipyfilechooser` for a clickable file navigator.\n")
                print("For now, type a full .ipynb path below and press Enter.\n")

                text = widgets.Text(
                    value=str(selected_nb["path"]),
                    description='Notebook:',
                    layout=widgets.Layout(width='100%')
                )

                def on_text_submit(change):
                    p = Path(change["new"]).expanduser()
                    selected_nb["path"] = p
                    update_label()

                text.on_submit(on_text_submit)
                display(text)

    browse_button.on_click(on_browse_click)

    def on_confirm(b):
        done_flag["done"] = True

    confirm_button.on_click(on_confirm)

    # initialize label
    update_label()

    ui = widgets.VBox([
        dropdown,
        widgets.HBox([browse_button, confirm_button]),
        path_label,
        out
    ])
    display(ui)

    # wait until user clicks Confirm
    while not done_flag["done"]:
        time.sleep(0.1)

    print(f"\n✔ Using notebook:\n  {selected_nb['path']}\n")
    return selected_nb["path"]


# ─────────────────────────────────────────────
# INTERACTIVE OUTPUT DIRECTORY CHOOSER
# ─────────────────────────────────────────────

def choose_output_path_interactive(default_dir: Path, default_name: str, repo_root: Path) -> Path:
    """
    Interactive directory chooser using widgets.

    - Dropdown with default dir, repo root, cwd.
    - 'Browse…' to pick a directory (ipyfilechooser if available).
    - 'Confirm' to finalize.
    """
    import ipywidgets as widgets
    from IPython.display import display

    default_dir = default_dir.resolve()
    repo_root = repo_root.resolve()

    choices = [
        ("Default (next to source)", default_dir),
        ("Repo root", repo_root),
        ("Current working dir", Path.cwd().resolve()),
    ]
    options = {label: path for label, path in choices}

    dropdown = widgets.Dropdown(
        options=options,
        description='Save dir:',
        layout=widgets.Layout(width='100%')
    )

    browse_button = widgets.Button(description='Browse…')
    confirm_button = widgets.Button(description='Confirm', button_style='success')
    path_label = widgets.Label()
    out = widgets.Output()

    selected_dir = {"path": default_dir}
    done_flag = {"done": False}

    def update_label():
        path_label.value = f"Output will be: {(selected_dir['path'] / default_name).resolve()}"

    def on_dropdown_change(change):
        if change["name"] == "value":
            selected_dir["path"] = change["new"]
            update_label()

    dropdown.observe(on_dropdown_change)

    def on_browse_click(b):
        out.clear_output()
        with out:
            try:
                from ipyfilechooser import FileChooser

                fc = FileChooser(str(repo_root))
                fc.title = 'Pick output directory'
                fc.use_dir_icons = True
                fc.show_only_dirs = True

                def on_done(chooser):
                    if chooser.selected_path is None:
                        return
                    selected_dir["path"] = Path(chooser.selected_path)
                    update_label()

                fc.register_callback(on_done)
                display(fc)

            except ImportError:
                print("ipyfilechooser not installed.")
                print("Run `pip install ipyfilechooser` for a clickable directory navigator,\n"
                      "or type a directory path below and press Enter.\n")

                text = widgets.Text(
                    value=str(selected_dir["path"]),
                    description='Dir:',
                    layout=widgets.Layout(width='100%')
                )

                def on_text_submit(change):
                    p = Path(change["new"]).expanduser()
                    selected_dir["path"] = p
                    update_label()

                text.on_submit(on_text_submit)
                display(text)

    browse_button.on_click(on_browse_click)

    def on_confirm(b):
        done_flag["done"] = True

    confirm_button.on_click(on_confirm)

    update_label()

    ui = widgets.VBox([
        dropdown,
        widgets.HBox([browse_button, confirm_button]),
        path_label,
        out
    ])
    display(ui)

    while not done_flag["done"]:
        time.sleep(0.1)

    out_path = (selected_dir["path"] / default_name).resolve()
    out_path.parent.mkdir(parents=True, exist_ok=True)
    print(f"\n✔ Output will be saved to:\n  {out_path}\n")
    return out_path


# ─────────────────────────────────────────────
# Section parsing utilities
# ─────────────────────────────────────────────

def parse_label(label: str, max_depth: int = MAX_DEPTH):
    """Turn '2.5.1' into (2,5,1,0) for numeric comparison."""
    parts = [int(p) for p in label.split(".")]
    if len(parts) < max_depth:
        parts += [0] * (max_depth - len(parts))
    return tuple(parts[:max_depth])


def label_in_range(label: str, start: str, end: str) -> bool:
    t = parse_label(label)
    s = parse_label(start)
    e = parse_label(end)
    return s <= t <= e


# ─────────────────────────────────────────────
# MAIN FLOW
# ─────────────────────────────────────────────

cwd = Path.cwd()
repo_root = find_repo_root(cwd)
print("📂 Repo root:", repo_root)

matches = find_notebooks(repo_root, SOURCE_NOTEBOOK_NAME)

if not matches:
    raise FileNotFoundError(f"❌ No match found for {SOURCE_NOTEBOOK_NAME} in repo.")

# Interactive notebook selection (even if there's just one, you can still confirm it)
nb_path = choose_notebook_interactive(matches, repo_root)

# Default output name
default_output_name = (
    f"{nb_path.stem}_sections_"
    f"{RANGE_START_LABEL.replace('.', '-')}_to_{RANGE_END_LABEL.replace('.', '-')}.ipynb"
)

# Interactive output dir selection
out_path = choose_output_path_interactive(nb_path.parent, default_output_name, repo_root)

# Load source notebook
with nb_path.open("r", encoding="utf-8") as f:
    nb = json.load(f)

cells = nb.get("cells", [])

# Match headings like "# 2.5", "## 2.5.1", "### 7.2.3.1"
section_re = re.compile(r"^\s*#{1,6}\s*(\d+(?:\.\d+){0,3})\b")

current_label = None
seen_labels = set()
collected = []

for idx, cell in enumerate(cells):
    ctype = cell.get("cell_type")
    src = cell.get("source", [])

    if ctype == "markdown" and src:
        for line in src:
            m = section_re.match(line)
            if m:
                current_label = m.group(1)
                seen_labels.add(current_label)
                break
        continue

    if ctype == "code" and current_label is not None:
        if label_in_range(current_label, RANGE_START_LABEL, RANGE_END_LABEL):
            collected.append({
                "cell_type": "code",
                "metadata": {},
                "source": src,
                "outputs": [],
                "execution_count": None
            })

# Build and write new notebook
new_nb = {
    "cells": [
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "# Aggregated Section Range\n",
                f"Range: {RANGE_START_LABEL} → {RANGE_END_LABEL} (inclusive)\n\n",
                f"Section labels detected in source: {sorted(seen_labels)}\n",
                f"Selected notebook: `{nb_path}`\n",
                f"Output notebook: `{out_path}`\n"
            ]
        }
    ] + collected,
    "metadata": {
        "kernelspec": nb["metadata"].get("kernelspec", {}),
        "language_info": nb["metadata"].get("language_info", {})
    },
    "nbformat": 4,
    "nbformat_minor": nb.get("nbformat_minor", 5)
}

with out_path.open("w", encoding="utf-8") as f:
    json.dump(new_nb, f, indent=2)

print(f"\n✅ Done.")
print(f"   Range: {RANGE_START_LABEL} → {RANGE_END_LABEL}")
print(f"   Labels seen: {sorted(seen_labels)}")
print(f"   Included {len(collected)} code cells.")
print(f"   Output: {out_path}")


In [2]:
# INLINE v8: Find 01_EDA.ipynb in repo, ask which one to use,
# then aggregate a section range into a new .ipynb and ask where to save it.

import json, re
from pathlib import Path

# ─────────────────────────────────────────────
# CONFIG — EDIT ONLY THESE
# ─────────────────────────────────────────────
RANGE_START_LABEL = "2.5"
RANGE_END_LABEL   = "7.2"

SOURCE_NOTEBOOK_NAME = "02_DQ_IF.ipynb"
MAX_DEPTH = 4
# ─────────────────────────────────────────────


# ─────────────────────────────────────────────
# Repo discovery + search / choose helpers
# ─────────────────────────────────────────────

def find_repo_root(start: Path) -> Path:
    """Walk upward from start to find a folder containing .git."""
    current = start.resolve()
    for parent in [current] + list(current.parents):
        if (parent / ".git").is_dir():
            return parent
    return current


def find_notebooks(repo_root: Path, name: str):
    """Return all matching paths inside the repo."""
    return list(repo_root.rglob(name))


def extract_level_num(path: Path) -> int:
    """Extract numeric Level_# from path; non-matches get huge number."""
    m = re.search(r"Level_(\d+)", str(path))
    return int(m.group(1)) if m else 999999


def choose_notebook(matches):
    """Ask user which notebook to use, with the input box pushed down."""
    # sort matches by numeric level
    matches = sorted(matches, key=extract_level_num)

    print("\n📄 Multiple notebooks found:\n")
    for i, m in enumerate(matches):
        print(f"  [{i}] {m}")

    # push input widget down so it doesn't cover the list
    print("\n" * 5)
    import time
    time.sleep(0.2)

    while True:
        choice = input("👉 Enter the number of the notebook you want to use: ").strip()
        if choice.isdigit():
            idx = int(choice)
            if 0 <= idx < len(matches):
                print(f"\n✔ Using: {matches[idx]}\n")
                return matches[idx]
        print("❌ Invalid choice, try again.\n")


# ─────────────────────────────────────────────
# Section parsing utilities
# ─────────────────────────────────────────────

def parse_label(label: str, max_depth: int = MAX_DEPTH):
    """Turn '2.5.1' into (2,5,1,0) for numeric comparison."""
    parts = [int(p) for p in label.split(".")]
    if len(parts) < max_depth:
        parts += [0] * (max_depth - len(parts))
    return tuple(parts[:max_depth])


def label_in_range(label: str, start: str, end: str) -> bool:
    t = parse_label(label)
    s = parse_label(start)
    e = parse_label(end)
    return s <= t <= e


# ─────────────────────────────────────────────
# Output path chooser
# ─────────────────────────────────────────────

def choose_output_path_interactive(default_dir: Path, default_name: str, repo_root: Path) -> Path:
    """
    Interactive directory chooser using widgets.

    - Shows a dropdown of common dirs (default, repo root, cwd).
    - 'Browse...' button opens a directory navigator (if ipyfilechooser is installed),
      or a text box fallback.
    - 'Confirm' button finalizes the selection.
    """
    import ipywidgets as widgets
    from IPython.display import display
    import time

    default_dir = default_dir.resolve()
    repo_root = repo_root.resolve()

    # ---- initial options you can click on ----
    choices = [
        ("Default (next to source)", default_dir),
        ("Repo root", repo_root),
        ("Current working dir", Path.cwd().resolve()),
    ]
    options = {label: path for label, path in choices}

    dropdown = widgets.Dropdown(
        options=options,
        description='Save dir:',
        layout=widgets.Layout(width='100%')
    )

    browse_button = widgets.Button(description='Browse…')
    confirm_button = widgets.Button(description='Confirm', button_style='success')
    path_label = widgets.Label()
    out = widgets.Output()

    selected_dir = {"path": default_dir}
    done_flag = {"done": False}

    def update_label():
        path_label.value = f"Selected: {(selected_dir['path'] / default_name).resolve()}"

    def on_dropdown_change(change):
        if change["name"] == "value":
            selected_dir["path"] = change["new"]
            update_label()

    dropdown.observe(on_dropdown_change)

    def on_browse_click(b):
        out.clear_output()
        with out:
            try:
                # Optional: directory navigator with click support
                from ipyfilechooser import FileChooser  # pip install ipyfilechooser

                fc = FileChooser(str(repo_root))
                fc.title = 'Pick output directory'
                fc.use_dir_icons = True
                fc.show_only_dirs = True

                def on_done(chooser):
                    selected_dir["path"] = Path(chooser.selected_path)
                    update_label()

                fc.register_callback(on_done)
                display(fc)

            except ImportError:
                print("ipyfilechooser not installed.")
                print("Run `pip install ipyfilechooser` for a clickable navigator,")
                print("or type a directory path below and press Enter.\n")

                text = widgets.Text(
                    value=str(selected_dir["path"]),
                    description='Dir:',
                    layout=widgets.Layout(width='100%')
                )

                def on_text_submit(change):
                    p = Path(change["new"]).expanduser()
                    selected_dir["path"] = p
                    update_label()

                text.on_submit(on_text_submit)
                display(text)

    browse_button.on_click(on_browse_click)

    def on_confirm(b):
        done_flag["done"] = True

    confirm_button.on_click(on_confirm)

    # initialize label
    update_label()

    ui = widgets.VBox([
        dropdown,
        widgets.HBox([browse_button, confirm_button]),
        path_label,
        out
    ])
    display(ui)

    # wait until user clicks Confirm
    while not done_flag["done"]:
        time.sleep(0.1)

    out_path = (selected_dir["path"] / default_name).resolve()
    out_path.parent.mkdir(parents=True, exist_ok=True)
    print(f"\n✔ Output will be saved to:\n  {out_path}\n")
    return out_path


# def choose_output_path(default_dir: Path, default_name: str) -> Path:
    """
    Ask the user where to save the output notebook.

    - Press Enter → use default_dir/default_name
    - Enter a directory path → save as that_dir/default_name
    - Enter a path ending in .ipynb → use that exact file path
    """
    default_path = default_dir / default_name
    print(f"💾 Default output path:\n  {default_path}")
    print("You can:")
    print("  • Press Enter to accept the default, or")
    print("  • Enter a directory path, or")
    print("  • Enter a full file path ending in .ipynb\n")

    user_input = input("📂 Output path (optional): ").strip()

    if not user_input:
        out_path = default_path
    else:
        p = Path(user_input).expanduser()
        # If user typed a file (ends with .ipynb), use it directly
        if p.suffix == ".ipynb":
            out_path = p
        else:
            # Treat as directory; append default_name
            out_path = p / default_name

    out_path = out_path.resolve()
    out_path.parent.mkdir(parents=True, exist_ok=True)
    print(f"\n✔ Output will be saved to:\n  {out_path}\n")
    return out_path


# ─────────────────────────────────────────────
# Locate repo + pick notebook
# ─────────────────────────────────────────────

cwd = Path.cwd()
repo_root = find_repo_root(cwd)
print("📂 Repo root:", repo_root)

matches = find_notebooks(repo_root, SOURCE_NOTEBOOK_NAME)

if not matches:
    raise FileNotFoundError(f"❌ No match found for {SOURCE_NOTEBOOK_NAME} in repo.")

if len(matches) == 1:
    nb_path = matches[0]
    print(f"✔ Found notebook: {nb_path}")
else:
    nb_path = choose_notebook(matches)

# default output naming
default_output_name = (
    f"{nb_path.stem}_sections_"
    f"{RANGE_START_LABEL.replace('.', '-')}_to_{RANGE_END_LABEL.replace('.', '-')}.ipynb"
)

# ask user where to save
out_path = choose_output_path(nb_path.parent, default_output_name)


# ─────────────────────────────────────────────
# Load notebook + aggregate cells
# ─────────────────────────────────────────────

with nb_path.open("r", encoding="utf-8") as f:
    nb = json.load(f)

cells = nb.get("cells", [])

section_re = re.compile(r"^\s*#{1,6}\s*(\d+(?:\.\d+){0,3})\b")

current_label = None
seen_labels = set()
collected = []

for idx, cell in enumerate(cells):
    ctype = cell.get("cell_type")
    src = cell.get("source", [])

    # Find section headers anywhere inside markdown
    if ctype == "markdown" and src:
        for line in src:
            m = section_re.match(line)
            if m:
                current_label = m.group(1)
                seen_labels.add(current_label)
                break
        continue

    # Collect code cells
    if ctype == "code" and current_label is not None:
        if label_in_range(current_label, RANGE_START_LABEL, RANGE_END_LABEL):
            collected.append({
                "cell_type": "code",
                "metadata": {},
                "source": src,
                "outputs": [],
                "execution_count": None
            })


# ─────────────────────────────────────────────
# Build + write new notebook
# ─────────────────────────────────────────────

new_nb = {
    "cells": [
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "# Aggregated Section Range\n",
                f"Range: {RANGE_START_LABEL} → {RANGE_END_LABEL} (inclusive)\n\n",
                f"Section labels detected in source: {sorted(seen_labels)}\n",
                f"Selected notebook: `{nb_path}`\n",
                f"Output notebook: `{out_path}`\n"
            ]
        }
    ] + collected,
    "metadata": {
        "kernelspec": nb["metadata"].get("kernelspec", {}),
        "language_info": nb["metadata"].get("language_info", {})
    },
    "nbformat": 4,
    "nbformat_minor": nb.get("nbformat_minor", 5)
}

with out_path.open("w", encoding="utf-8") as f:
    json.dump(new_nb, f, indent=2)

print(f"\n✅ Done.")
print(f"   Range: {RANGE_START_LABEL} → {RANGE_END_LABEL}")
print(f"   Labels seen: {sorted(seen_labels)}")
print(f"   Included {len(collected)} code cells.")
print(f"   Output: {out_path}")


📂 Repo root: /Users/b/DATA/PROJECTS/Telco
✔ Found notebook: /Users/b/DATA/PROJECTS/Telco/Level_3/notebooks/02_DQ_IF.ipynb


NameError: name 'choose_output_path' is not defined

In [ ]:
# INLINE v7: Find 01_EDA.ipynb anywhere in the repo, ask user to choose match,
# then aggregate a numeric section range into a new .ipynb.

import json, re
from pathlib import Path

# ─────────────────────────────────────────────
# CONFIG — EDIT ONLY THESE TWO
# ─────────────────────────────────────────────
RANGE_START_LABEL = "2.0"
RANGE_END_LABEL   = "2.4"

SOURCE_NOTEBOOK_NAME = "01_EDA.ipynb"
MAX_DEPTH = 4
# ─────────────────────────────────────────────


# ─────────────────────────────────────────────
# Repo discovery + search functions
# ─────────────────────────────────────────────

def find_repo_root(start: Path) -> Path:
    """Walk upward from start to find a folder containing .git."""
    current = start.resolve()
    for parent in [current] + list(current.parents):
        if (parent / ".git").is_dir():
            return parent
    return current


def find_notebooks(repo_root: Path, name: str):
    """Return all matching paths inside the repo."""
    return list(repo_root.rglob(name))

def choose_notebook(matches):
    """v2. Ask user which notebook to use, but push the input box down so it doesn't cover the list."""
    print("\n📄 Multiple notebooks found:\n")
    for i, m in enumerate(matches):
        print(f"  [{i}] {m}")

    # MOVE THE INPUT BOX DOWN
    print("\n" * 50)   # add vertical space so popup appears lower

    # OPTIONAL: make sure output renders before the prompt
    import time
    time.sleep(0.2)

    # Now ask the user
    while True:
        choice = input("👉 Enter the number of the notebook you want to use: ").strip()
        if choice.isdigit():
            idx = int(choice)
            if 0 <= idx < len(matches):
                print(f"\n✔ Using: {matches[idx]}\n")
                return matches[idx]
        print("❌ Invalid choice, try again.\n")

# def choose_notebook(matches):
    """v1. Ask user which notebook to use."""
    print("\n📄 Multiple notebooks found:")
    for i, m in enumerate(matches):
        print(f"  [{i}] {m}")

    while True:
        choice = input("\n👉 Enter the number of the notebook you want to use: ").strip()
        if choice.isdigit():
            idx = int(choice)
            if 0 <= idx < len(matches):
                print(f"✔ Using: {matches[idx]}\n")
                return matches[idx]
        print("❌ Invalid choice, try again.")


# ─────────────────────────────────────────────
# Section parsing utilities
# ─────────────────────────────────────────────

def parse_label(label: str, max_depth: int = MAX_DEPTH):
    """Turn '2.5.1' into (2,5,1,0)."""
    parts = [int(p) for p in label.split(".")]
    if len(parts) < max_depth:
        parts += [0] * (max_depth - len(parts))
    return tuple(parts[:max_depth])


def label_in_range(label: str, start: str, end: str) -> bool:
    t = parse_label(label)
    s = parse_label(start)
    e = parse_label(end)
    return s <= t <= e


# ─────────────────────────────────────────────
# Locate repo + pick notebook
# ─────────────────────────────────────────────

cwd = Path.cwd()
repo_root = find_repo_root(cwd)
print("📂 Repo root:", repo_root)

import re

def extract_level_num(path: Path) -> int:
    """
    Find something like 'Level_4' or 'Level_10' in the path.
    If not found, return a huge number so those go last.
    """
    m = re.search(r"Level_(\d+)", str(path))
    if m:
        return int(m.group(1))
    return 999999  # lowest priority for non-level folders

# Sort matches x numeric level
matches = sorted(matches, key=extract_level_num)

# Sort matches x lexicographic
# matches = find_notebooks(repo_root, SOURCE_NOTEBOOK_NAME)

if not matches:
    raise FileNotFoundError(f"❌ No match found for {SOURCE_NOTEBOOK_NAME} in repo.")

if len(matches) == 1:
    nb_path = matches[0]
    print(f"✔ Found notebook: {nb_path}")
else:
    nb_path = choose_notebook(matches)

# Output path
output_name = f"{nb_path.stem}_sections_{RANGE_START_LABEL.replace('.', '-')}_to_{RANGE_END_LABEL.replace('.', '-')}.ipynb"
out_path = nb_path.parent / output_name

# ─────────────────────────────────────────────
# Load notebook + aggregate cells
# ─────────────────────────────────────────────

with nb_path.open("r", encoding="utf-8") as f:
    nb = json.load(f)

cells = nb.get("cells", [])

section_re = re.compile(r"^\s*#{1,6}\s*(\d+(?:\.\d+){0,3})\b")

current_label = None
seen_labels = set()
collected = []

for idx, cell in enumerate(cells):
    ctype = cell.get("cell_type")
    src = cell.get("source", [])

    # Find section headers anywhere inside markdown
    if ctype == "markdown" and src:
        for line in src:
            m = section_re.match(line)
            if m:
                current_label = m.group(1)
                seen_labels.add(current_label)
                break
        continue

    # Collect code cells
    if ctype == "code" and current_label is not None:
        if label_in_range(current_label, RANGE_START_LABEL, RANGE_END_LABEL):
            collected.append({
                "cell_type": "code",
                "metadata": {},
                "source": src,
                "outputs": [],
                "execution_count": None
            })


# ─────────────────────────────────────────────
# Build + write new notebook
# ─────────────────────────────────────────────

new_nb = {
    "cells": [
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "# Aggregated Section Range\n",
                f"Range: {RANGE_START_LABEL} → {RANGE_END_LABEL} (inclusive)\n\n",
                f"Section labels detected in source: {sorted(seen_labels)}\n",
                f"Selected notebook: `{nb_path}`\n"
            ]
        }
    ] + collected,
    "metadata": {
        "kernelspec": nb["metadata"].get("kernelspec", {}),
        "language_info": nb["metadata"].get("language_info", {})
    },
    "nbformat": 4,
    "nbformat_minor": nb.get("nbformat_minor", 5)
}

with out_path.open("w", encoding="utf-8") as f:
    json.dump(new_nb, f, indent=2)

print(f"\n✅ Done.")
print(f"   Range: {RANGE_START_LABEL} → {RANGE_END_LABEL}")
print(f"   Labels seen: {sorted(seen_labels)}")
print(f"   Included {len(collected)} code cells.")
print(f"   Output: {out_path}")


In [ ]:
# Render one collapsible cell per each python cell in section 2
import json
import html
from pathlib import Path
from IPython.display import HTML, display

# ---------------------------------------------------------
# 1️⃣  CONFIGURATION
# ---------------------------------------------------------
NOTEBOOK_PATH = Path("01_EDA.ipynb")  # change if your notebook has a different name

if not NOTEBOOK_PATH.exists():
    raise FileNotFoundError(f"Notebook not found: {NOTEBOOK_PATH.resolve()}")

# ---------------------------------------------------------
# 2️⃣  LOAD NOTEBOOK
# ---------------------------------------------------------
with open(NOTEBOOK_PATH, "r", encoding="utf-8") as f:
    nb = json.load(f)

# ---------------------------------------------------------
# 3️⃣  FIND CODE CELLS STARTING WITH "# 2"
# ---------------------------------------------------------
cells_info = []  # list of dicts: {title, code}

for idx, cell in enumerate(nb.get("cells", []), start=1):
    if cell.get("cell_type") != "code":
        continue

    src = "".join(cell.get("source", []))
    if not src.strip():
        continue

    # First non-empty line
    first_line = next((ln for ln in src.splitlines() if ln.strip()), "")
    if not first_line.strip().startswith("# 2"):
        continue

    # Title from first line (strip leading "#")
    raw_title = first_line.lstrip("#").strip()
    title = raw_title or f"Section 2 cell #{idx}"

    cells_info.append(
        {
            "title": title,
            "code": src.rstrip(),
            "index": idx,
        }
    )

# ---------------------------------------------------------
# 4️⃣  RENDER ONE COLLAPSIBLE PANEL PER MATCHING CELL
# ---------------------------------------------------------
if not cells_info:
    display(HTML(
        "<p style='color:#b91c1c;font-family:-apple-system,BlinkMacSystemFont,\"Segoe UI\",sans-serif;'>"
        "⚠️ No code cells starting with <code># 2</code> were found in this notebook."
        "</p>"
    ))
else:
    panels = []
    for i, info in enumerate(cells_info, start=1):
        title = html.escape(info["title"])
        code  = html.escape(info["code"])

        panel_html = f"""
<details style="
  margin:10px 0;
  border:1px solid #e5e7eb;
  border-radius:10px;
  background:linear-gradient(135deg,#f9fafb,#eef2ff);
  font-family:-apple-system,BlinkMacSystemFont,'Segoe UI',sans-serif;
">
  <summary style="
    cursor:pointer;
    list-style:none;
    padding:8px 12px;
    display:flex;
    align-items:center;
    justify-content:space-between;
    gap:8px;
    font-weight:600;
    color:#0f172a;
  ">
    <span style="display:inline-flex;align-items:center;gap:6px;">
      📜 <span>{i}. {title}</span>
    </span>
    <span style="
      font-size:11px;
      padding:2px 8px;
      border-radius:999px;
      background:#0f172a;
      color:#f9fafb;
      text-transform:uppercase;
      letter-spacing:0.06em;
    ">
      toggle
    </span>
  </summary>

  <pre style="
    margin:0;
    padding:10px 14px;
    background:#020617;
    color:#e5e7eb;
    font-family:'Fira Code',ui-monospace,Consolas,'Courier New',monospace;
    font-size:13px;
    line-height:1.55;
    white-space:pre-wrap;
    word-break:break-word;
    border-radius:0 0 10px 10px;
    border-top:1px solid #1f2937;
  ">{code}</pre>
</details>
"""
        panels.append(panel_html)

    wrapper = f"""
<div style="margin:12px 0;">
  <div style="
    font-family:-apple-system,BlinkMacSystemFont,'Segoe UI',sans-serif;
    font-size:13px;
    color:#4b5563;
    margin-bottom:6px;
  ">
    Found <strong>{len(cells_info)}</strong> code cell(s) starting with <code># 2</code> in <code>{NOTEBOOK_PATH.name}</code>:
  </div>
  {''.join(panels)}
</div>
"""
    display(HTML(wrapper))


In [ ]:
import json
import html
from pathlib import Path
from IPython.display import HTML, display

# ---------------------------------------------------------
# 1️⃣  CONFIGURATION
# ---------------------------------------------------------
NOTEBOOK_PATH = Path("01_EDA.ipynb")  # change if your notebook has a different name

if not NOTEBOOK_PATH.exists():
    raise FileNotFoundError(f"Notebook not found: {NOTEBOOK_PATH.resolve()}")

# ---------------------------------------------------------
# 2️⃣  LOAD NOTEBOOK
# ---------------------------------------------------------
with open(NOTEBOOK_PATH, "r", encoding="utf-8") as f:
    nb = json.load(f)

# ---------------------------------------------------------
# 3️⃣  FIND CODE CELLS STARTING WITH "# 2"
# ---------------------------------------------------------
cells_info = []  # list of dicts: {title, code}

for idx, cell in enumerate(nb.get("cells", []), start=1):
    if cell.get("cell_type") != "code":
        continue

    src = "".join(cell.get("source", []))
    if not src.strip():
        continue

    # First non-empty line
    first_line = next((ln for ln in src.splitlines() if ln.strip()), "")
    if not first_line.strip().startswith("# 2"):
        continue

    # Title from first line (strip leading "#")
    raw_title = first_line.lstrip("#").strip()
    title = raw_title or f"Section 2 cell #{idx}"

    cells_info.append(
        {
            "title": title,
            "code": src.rstrip(),
            "index": idx,
        }
    )

# ---------------------------------------------------------
# 4️⃣  RENDER ONE COLLAPSIBLE PANEL PER MATCHING CELL
# ---------------------------------------------------------
if not cells_info:
    display(HTML(
        "<p style='color:#b91c1c;font-family:-apple-system,BlinkMacSystemFont,\"Segoe UI\",sans-serif;'>"
        "⚠️ No code cells starting with <code># 2</code> were found in this notebook."
        "</p>"
    ))
else:
    panels = []
    for i, info in enumerate(cells_info, start=1):
        title = html.escape(info["title"])
        code  = html.escape(info["code"])

        panel_html = f"""
<details style="
  margin:10px 0;
  border:1px solid #e5e7eb;
  border-radius:10px;
  background:linear-gradient(135deg,#f9fafb,#eef2ff);
  font-family:-apple-system,BlinkMacSystemFont,'Segoe UI',sans-serif;
">
  <summary style="
    cursor:pointer;
    list-style:none;
    padding:8px 12px;
    display:flex;
    align-items:center;
    justify-content:space-between;
    gap:8px;
    font-weight:600;
    color:#0f172a;
  ">
    <span style="display:inline-flex;align-items:center;gap:6px;">
      📜 <span>{i}. {title}</span>
    </span>
    <span style="
      font-size:11px;
      padding:2px 8px;
      border-radius:999px;
      background:#0f172a;
      color:#f9fafb;
      text-transform:uppercase;
      letter-spacing:0.06em;
    ">
      toggle
    </span>
  </summary>

  <pre style="
    margin:0;
    padding:10px 14px;
    background:#020617;
    color:#e5e7eb;
    font-family:'Fira Code',ui-monospace,Consolas,'Courier New',monospace;
    font-size:13px;
    line-height:1.55;
    white-space:pre-wrap;
    word-break:break-word;
    border-radius:0 0 10px 10px;
    border-top:1px solid #1f2937;
  ">{code}</pre>
</details>
"""
        panels.append(panel_html)

    wrapper = f"""
<div style="margin:12px 0;">
  <div style="
    font-family:-apple-system,BlinkMacSystemFont,'Segoe UI',sans-serif;
    font-size:13px;
    color:#4b5563;
    margin-bottom:6px;
  ">
    Found <strong>{len(cells_info)}</strong> code cell(s) starting with <code># 2</code> in <code>{NOTEBOOK_PATH.name}</code>:
  </div>
  {''.join(panels)}
</div>
"""
    display(HTML(wrapper))


In [ ]:
import json
from pathlib import Path
from IPython.display import HTML, display

# ---------------------------------------------------------
# 1️⃣  CONFIGURATION  — edit this if your file is named differently
# ---------------------------------------------------------
NOTEBOOK_PATH = Path("01_EDA.ipynb")

# ---------------------------------------------------------
# 2️⃣  LOAD NOTEBOOK
# ---------------------------------------------------------
if not NOTEBOOK_PATH.exists():
    raise FileNotFoundError(f"Notebook not found: {NOTEBOOK_PATH.resolve()}")

with open(NOTEBOOK_PATH, "r", encoding="utf-8") as f:
    nb = json.load(f)

# ---------------------------------------------------------
# 3️⃣  GATHER CODE CELLS THAT START WITH "# 2"
# ---------------------------------------------------------
section_code = []
for cell in nb.get("cells", []):
    if cell.get("cell_type") == "code":
        src = "".join(cell.get("source", []))
        if not src.strip():
            continue
        first_line = next((ln.strip() for ln in src.splitlines() if ln.strip()), "")
        if first_line.startswith("# 2"):
            section_code.append(src.rstrip())

# ---------------------------------------------------------
# 4️⃣  BUILD COLLAPSIBLE HTML
# ---------------------------------------------------------
if section_code:
    combined = "\n\n".join(section_code)
    html = f"""
<details style='margin:12px 0; border:1px solid #e5e7eb; border-radius:10px;
  background:linear-gradient(135deg,#f9fafb,#eef2ff); font-family:-apple-system,BlinkMacSystemFont,"Segoe UI",sans-serif;'>
  <summary style='cursor:pointer; list-style:none; padding:10px 14px; font-weight:600;
    color:#0f172a; display:flex; justify-content:space-between; align-items:center;'>
    <span style='display:flex;align-items:center;gap:8px;'>📜 Aggregated Section 2 Code ({len(section_code)} cells)</span>
    <span style='font-size:11px; background:#0f172a; color:#f9fafb; padding:2px 8px;
      border-radius:999px; text-transform:uppercase; letter-spacing:0.06em;'>click to toggle</span>
  </summary>

  <pre style='margin:0; padding:12px 16px; background:#020617; color:#e5e7eb;
    font-family:"Fira Code",ui-monospace,Consolas,"Courier New",monospace;
    font-size:13px; line-height:1.55; white-space:pre-wrap; word-break:break-word;
    border-radius:0 0 10px 10px; border-top:1px solid #1f2937;'>
{combined}
  </pre>
</details>
"""
    display(HTML(html))
else:
    display(HTML("<p style='color:#b91c1c;'>⚠️ No code cells starting with '# 2' found in this notebook.</p>"))


In [ ]:

import json
from pathlib import Path
from IPython.display import Markdown, display

# --- identify current notebook file ---
NOTEBOOK_PATH = Path("01_EDA.ipynb")  # adjust if your notebook name differs

# --- load the notebook ---
with open(NOTEBOOK_PATH, "r", encoding="utf-8") as f:
    nb = json.load(f)

# --- gather only code cells whose first nonblank line starts with '# 2' ---
matching_cells = []
for cell in nb.get("cells", []):
    if cell.get("cell_type") == "code":
        src = "".join(cell.get("source", []))
        first_line = next((ln.strip() for ln in src.splitlines() if ln.strip()), "")
        if first_line.startswith("# 2"):
            matching_cells.append(src.strip())

# --- aggregate ---
if matching_cells:
    header = (
        "```python\n"
        "# ==============================================\n"
        "# Aggregated Python Cells from 01_EDA.ipynb\n"
        "# Matching pattern: lines starting with '# 2'\n"
        "# ==============================================\n\n"
    )
    footer = "\n```"
    joined_code = "\n\n\n".join(matching_cells)
    display(Markdown(header + joined_code + footer))
else:
    display(Markdown("> ⚠️ **No code cells starting with `# 2` were found.**"))


In [ ]:
import json, io, os
from pathlib import Path
from IPython.display import Markdown, display

# ---------------------------------------------------------------------
# 1️⃣ Try to infer current notebook name (works in most Jupyter setups)
# ---------------------------------------------------------------------
try:
    from IPython import get_ipython
    ip = get_ipython()
    if ip is not None and hasattr(ip, 'kernel'):
        connection_file = Path(ip.kernel.connection_file)
        notebook_name = next(
            (p.name for p in Path('.').glob('*.ipynb') if str(p.stat().st_ino) in str(connection_file)),
            None
        )
    else:
        notebook_name = None
except Exception:
    notebook_name = None

# fallback manual name if automatic detection fails
if not notebook_name:
    notebook_name = "01_EDA.ipynb"

NOTEBOOK_PATH = Path(notebook_name)

# ---------------------------------------------------------------------
# 2️⃣ Load the notebook JSON
# ---------------------------------------------------------------------
if not NOTEBOOK_PATH.exists():
    raise FileNotFoundError(f"Notebook not found: {NOTEBOOK_PATH.resolve()}")

with open(NOTEBOOK_PATH, "r", encoding="utf-8") as f:
    nb = json.load(f)

# ---------------------------------------------------------------------
# 3️⃣ Collect code cells whose first non-empty line starts with "# 2"
# ---------------------------------------------------------------------
section_code = []
for cell in nb.get("cells", []):
    if cell.get("cell_type") == "code":
        src = "".join(cell.get("source", []))
        if not src.strip():
            continue
        first_line = next((ln.strip() for ln in src.splitlines() if ln.strip()), "")
        if first_line.startswith("# 2"):
            section_code.append(src.rstrip())

# ---------------------------------------------------------------------
# 4️⃣ Aggregate and render inline
# ---------------------------------------------------------------------
# if section_code:
#     combined = "\n\n".join(section_code)
#     md = f"```python\n{combined}\n```"
#     display(Markdown(f"### 🧩 Aggregated Section 2 Code ({len(section_code)} cells)\n\n" + md))
# else:
#     display(Markdown("> ⚠️ **No code cells starting with `# 2` found in this notebook.**"))

Path("section2_aggregate.py").write_text(combined, encoding="utf-8")
print("✅ Saved section2_aggregate.py")


In [ ]:
import json
from pathlib import Path
from IPython.display import HTML, display

# ---------------------------------------------------------
# 1️⃣  CONFIGURATION  — edit this if your file is named differently
# ---------------------------------------------------------
NOTEBOOK_PATH = Path("01_EDA.ipynb")

# ---------------------------------------------------------
# 2️⃣  LOAD NOTEBOOK
# ---------------------------------------------------------
if not NOTEBOOK_PATH.exists():
    raise FileNotFoundError(f"Notebook not found: {NOTEBOOK_PATH.resolve()}")

with open(NOTEBOOK_PATH, "r", encoding="utf-8") as f:
    nb = json.load(f)

# ---------------------------------------------------------
# 3️⃣  GATHER CODE CELLS THAT START WITH "# 2"
# ---------------------------------------------------------
section_code = []
for cell in nb.get("cells", []):
    if cell.get("cell_type") == "code":
        src = "".join(cell.get("source", []))
        if not src.strip():
            continue
        first_line = next((ln.strip() for ln in src.splitlines() if ln.strip()), "")
        if first_line.startswith("# 2"):
            section_code.append(src.rstrip())

# ---------------------------------------------------------
# 4️⃣  BUILD COLLAPSIBLE HTML
# ---------------------------------------------------------
if section_code:
    combined = "\n\n".join(section_code)
    html = f"""
<details style='margin:12px 0; border:1px solid #e5e7eb; border-radius:10px;
  background:linear-gradient(135deg,#f9fafb,#eef2ff); font-family:-apple-system,BlinkMacSystemFont,"Segoe UI",sans-serif;'>
  <summary style='cursor:pointer; list-style:none; padding:10px 14px; font-weight:600;
    color:#0f172a; display:flex; justify-content:space-between; align-items:center;'>
    <span style='display:flex;align-items:center;gap:8px;'>📜 Aggregated Section 2 Code ({len(section_code)} cells)</span>
    <span style='font-size:11px; background:#0f172a; color:#f9fafb; padding:2px 8px;
      border-radius:999px; text-transform:uppercase; letter-spacing:0.06em;'>click to toggle</span>
  </summary>

  <pre style='margin:0; padding:12px 16px; background:#020617; color:#e5e7eb;
    font-family:"Fira Code",ui-monospace,Consolas,"Courier New",monospace;
    font-size:13px; line-height:1.55; white-space:pre-wrap; word-break:break-word;
    border-radius:0 0 10px 10px; border-top:1px solid #1f2937;'>
{combined}
  </pre>
</details>
"""
    display(HTML(html))
else:
    display(HTML("<p style='color:#b91c1c;'>⚠️ No code cells starting with '# 2' found in this notebook.</p>"))


<details>
<summary style="
    cursor:pointer;background:#f7f7fb;border:2px solid #297be7ff;
padding:10px 12px;border-radius:10px;font-weight:700;">
extraction
</summary>

Perfect — you want a script that programmatically extracts all **Python code cells** from your `01_EDA.ipynb` Jupyter notebook that **begin with comments starting with `# 2`** (e.g., `# 2.0`, `# 2.3.1`, etc.), then aggregates them into one file or display block.

Here’s a clean, production-ready script that does exactly that 👇

---

### 🧩 `aggregate_section2_cells.py`

```python
#!/usr/bin/env python3
"""
aggregate_section2_cells.py
-------------------------------------
Extracts all Python code cells from 01_EDA.ipynb that start with "# 2"
and concatenates them into a single file named section2_cells.py
"""

import json
from pathlib import Path

# ------------------------------------------------------------------
# 1️⃣  CONFIGURATION
# ------------------------------------------------------------------
NOTEBOOK_PATH = Path("01_EDA.ipynb")   # adjust path if needed
OUTPUT_FILE   = Path("section2_cells.py")

# ------------------------------------------------------------------
# 2️⃣  VALIDATE INPUT FILE
# ------------------------------------------------------------------
if not NOTEBOOK_PATH.exists():
    raise FileNotFoundError(f"❌ Notebook not found: {NOTEBOOK_PATH.resolve()}")

# ------------------------------------------------------------------
# 3️⃣  LOAD NOTEBOOK
# ------------------------------------------------------------------
with open(NOTEBOOK_PATH, "r", encoding="utf-8") as f:
    nb = json.load(f)

# ------------------------------------------------------------------
# 4️⃣  FILTER CELLS THAT ARE PYTHON CODE STARTING WITH '# 2'
# ------------------------------------------------------------------
aggregated_code = []

for cell in nb.get("cells", []):
    if cell.get("cell_type") == "code":
        source = "".join(cell.get("source", []))
        # Check if the first non-blank line starts with '# 2'
        first_line = next((line.strip() for line in source.splitlines() if line.strip()), "")
        if first_line.startswith("# 2"):
            aggregated_code.append(source.strip())

# ------------------------------------------------------------------
# 5️⃣  WRITE TO OUTPUT FILE
# ------------------------------------------------------------------
if not aggregated_code:
    print("⚠️  No matching '# 2' cells found.")
else:
    header = (
        "# ==============================================\n"
        "# Aggregated Python Cells from 01_EDA.ipynb\n"
        "# Matching pattern: lines starting with '# 2'\n"
        "# ==============================================\n\n"
    )
    OUTPUT_FILE.write_text(header + "\n\n\n".join(aggregated_code), encoding="utf-8")
    print(f"✅ Extracted {len(aggregated_code)} matching cells to {OUTPUT_FILE.resolve()}")
```

---

### 💡 How it works

* Reads your `01_EDA.ipynb` as JSON.
* Filters only **code cells** (`cell_type == "code"`).
* Checks the **first non-empty line** — if it starts with `# 2`, it’s included.
* Joins them into a single Python file `section2_cells.py`.
* Keeps your notebook untouched.

---

### 🧰 Example usage

In your terminal (from the project root):

```bash
python aggregate_section2_cells.py
```

Then open `section2_cells.py` and you’ll see something like:

```python
# ==============================================
# Aggregated Python Cells from 01_EDA.ipynb
# Matching pattern: lines starting with '# 2'
# ==============================================

# 2.0 Data Quality Overview
print("Running Section 2.0 Checks...")

# 2.1 Base Schema & Consistency
...

# 2.2 Numeric Validation
...
```

---

Would you like the script to **output the aggregated code to the terminal instead of writing a file**, or to **append timestamps / section headers automatically** (so it’s easy to track runs)?
